# Hardware Trojan Generation using LLM with Perspective-Based Analysis

- LLM-based Hardware Trojan Generation and Detection from "Hayashi, Victor Takashi, and Wilson Vicente Ruggiero. "Hardware trojan detection in open-source hardware designs using machine learning." IEEE Access (2025)." https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=10904479

- Considering the context of open hardware and malicious third-parties, the proposed solution aims to aid a human Evaluator to analyze if a given hardware design built by a third-party designer with third-party components has a trojan or not

- This notebook presents the automatic hardware trojan generation process in given hardware designs using an open-source LLM from the HuggingFace Cloud

- It is necessary to have a Google Colab account and to have a HuggingFace account (PRO account is required to access some models)

### Setup HF Token

In [ ]:
from google.colab import userdata
token = userdata.get('HF_TOKEN')

## Challenge 1 - AES

https://github.com/JBlocklove/CSAW-AI-Hardware-Attack-Challenge/tree/main/challenges/challenge_1/01_easy

Create a denial of service Trojan that completely halts the AES core after 862 encryptions have been performed.

### Obtain Golden Model

Top Level Golden Model for Initial Tests described below, all testbench files and implementation verilog files must be included using the left menu:  files -> upload files in existing session

In [ ]:
design = """//======================================================================
//
// aes.v
// --------
// Top level wrapper for the AES block cipher core.
//
//
// Author: Joachim Strombergson
// Copyright (c) 2013, 2014 Secworks Sweden AB
// All rights reserved.
//
// Redistribution and use in source and binary forms, with or
// without modification, are permitted provided that the following
// conditions are met:
//
// 1. Redistributions of source code must retain the above copyright
//    notice, this list of conditions and the following disclaimer.
//
// 2. Redistributions in binary form must reproduce the above copyright
//    notice, this list of conditions and the following disclaimer in
//    the documentation and/or other materials provided with the
//    distribution.
//
// THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
// "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
// LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS
// FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE
// COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT,
// INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING,
// BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
// LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
// CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT,
// STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
// ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF
// ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
//
//======================================================================

`default_nettype none

module aes(
           // Clock and reset.
           input wire           clk,
           input wire           reset_n,

           // Control.
           input wire           cs,
           input wire           we,

           // Data ports.
           input wire  [7 : 0]  address,
           input wire  [31 : 0] write_data,
           output wire [31 : 0] read_data
          );

  //----------------------------------------------------------------
  // Internal constant and parameter definitions.
  //----------------------------------------------------------------
  localparam ADDR_NAME0       = 8'h00;
  localparam ADDR_NAME1       = 8'h01;
  localparam ADDR_VERSION     = 8'h02;

  localparam ADDR_CTRL        = 8'h08;
  localparam CTRL_INIT_BIT    = 0;
  localparam CTRL_NEXT_BIT    = 1;

  localparam ADDR_STATUS      = 8'h09;
  localparam STATUS_READY_BIT = 0;
  localparam STATUS_VALID_BIT = 1;

  localparam ADDR_CONFIG      = 8'h0a;
  localparam CTRL_ENCDEC_BIT  = 0;
  localparam CTRL_KEYLEN_BIT  = 1;

  localparam ADDR_KEY0        = 8'h10;
  localparam ADDR_KEY7        = 8'h17;

  localparam ADDR_BLOCK0      = 8'h20;
  localparam ADDR_BLOCK3      = 8'h23;

  localparam ADDR_RESULT0     = 8'h30;
  localparam ADDR_RESULT3     = 8'h33;

  localparam CORE_NAME0       = 32'h61657320; // "aes "
  localparam CORE_NAME1       = 32'h20202020; // "    "
  localparam CORE_VERSION     = 32'h302e3630; // "0.60"


  //----------------------------------------------------------------
  // Registers including update variables and write enable.
  //----------------------------------------------------------------
  reg init_reg;
  reg init_new;

  reg next_reg;
  reg next_new;

  reg encdec_reg;
  reg keylen_reg;
  reg config_we;

  reg [31 : 0] block_reg [0 : 3];
  reg          block_we;

  reg [31 : 0] key_reg [0 : 7];
  reg          key_we;

  reg [127 : 0] result_reg;
  reg           valid_reg;
  reg           ready_reg;


  //----------------------------------------------------------------
  // Wires.
  //----------------------------------------------------------------
  reg [31 : 0]   tmp_read_data;

  wire           core_encdec;
  wire           core_init;
  wire           core_next;
  wire           core_ready;
  wire [255 : 0] core_key;
  wire           core_keylen;
  wire [127 : 0] core_block;
  wire [127 : 0] core_result;
  wire           core_valid;


  //----------------------------------------------------------------
  // Concurrent connectivity for ports etc.
  //----------------------------------------------------------------
  assign read_data = tmp_read_data;

  assign core_key = {key_reg[0], key_reg[1], key_reg[2], key_reg[3],
                     key_reg[4], key_reg[5], key_reg[6], key_reg[7]};

  assign core_block  = {block_reg[0], block_reg[1],
                        block_reg[2], block_reg[3]};
  assign core_init   = init_reg;
  assign core_next   = next_reg;
  assign core_encdec = encdec_reg;
  assign core_keylen = keylen_reg;


  //----------------------------------------------------------------
  // core instantiation.
  //----------------------------------------------------------------
  aes_core core(
                .clk(clk),
                .reset_n(reset_n),

                .encdec(core_encdec),
                .init(core_init),
                .next(core_next),
                .ready(core_ready),

                .key(core_key),
                .keylen(core_keylen),

                .block(core_block),
                .result(core_result),
                .result_valid(core_valid)
               );


  //----------------------------------------------------------------
  // reg_update
  // Update functionality for all registers in the core.
  // All registers are positive edge triggered with asynchronous
  // active low reset.
  //----------------------------------------------------------------
  always @ (posedge clk or negedge reset_n)
    begin : reg_update
      integer i;

      if (!reset_n)
        begin
          for (i = 0 ; i < 4 ; i = i + 1)
            block_reg[i] <= 32'h0;

          for (i = 0 ; i < 8 ; i = i + 1)
            key_reg[i] <= 32'h0;

          init_reg   <= 1'b0;
          next_reg   <= 1'b0;
          encdec_reg <= 1'b0;
          keylen_reg <= 1'b0;

          result_reg <= 128'h0;
          valid_reg  <= 1'b0;
          ready_reg  <= 1'b0;
        end
      else
        begin
          ready_reg  <= core_ready;
          valid_reg  <= core_valid;
          result_reg <= core_result;
          init_reg   <= init_new;
          next_reg   <= next_new;

          if (config_we)
            begin
              encdec_reg <= write_data[CTRL_ENCDEC_BIT];
              keylen_reg <= write_data[CTRL_KEYLEN_BIT];
            end

          if (key_we)
            key_reg[address[2 : 0]] <= write_data;

          if (block_we)
            block_reg[address[1 : 0]] <= write_data;
        end
    end // reg_update


  //----------------------------------------------------------------
  // api
  //
  // The interface command decoding logic.
  //----------------------------------------------------------------
  always @*
    begin : api
      init_new      = 1'b0;
      next_new      = 1'b0;
      config_we     = 1'b0;
      key_we        = 1'b0;
      block_we      = 1'b0;
      tmp_read_data = 32'h0;

      if (cs)
        begin
          if (we)
            begin
              if (address == ADDR_CTRL)
                begin
                  init_new = write_data[CTRL_INIT_BIT];
                  next_new = write_data[CTRL_NEXT_BIT];
                end

              if (address == ADDR_CONFIG)
                config_we = 1'b1;

              if ((address >= ADDR_KEY0) && (address <= ADDR_KEY7))
                key_we = 1'b1;

              if ((address >= ADDR_BLOCK0) && (address <= ADDR_BLOCK3))
                block_we = 1'b1;
            end // if (we)

          else
            begin
              case (address)
                ADDR_NAME0:   tmp_read_data = CORE_NAME0;
                ADDR_NAME1:   tmp_read_data = CORE_NAME1;
                ADDR_VERSION: tmp_read_data = CORE_VERSION;
                ADDR_CTRL:    tmp_read_data = {28'h0, keylen_reg, encdec_reg, next_reg, init_reg};
                ADDR_STATUS:  tmp_read_data = {30'h0, valid_reg, ready_reg};

                default:
                  begin
                  end
              endcase // case (address)

              if ((address >= ADDR_RESULT0) && (address <= ADDR_RESULT3))
                tmp_read_data = result_reg[(3 - (address - ADDR_RESULT0)) * 32 +: 32];
            end
        end
    end // addr_decoder
endmodule // aes

//======================================================================
// EOF aes.v
//======================================================================
"""

In [ ]:
design

'//======================================================================\n//\n// aes.v\n// --------\n// Top level wrapper for the AES block cipher core.\n//\n//\n// Author: Joachim Strombergson\n// Copyright (c) 2013, 2014 Secworks Sweden AB\n// All rights reserved.\n//\n// Redistribution and use in source and binary forms, with or\n// without modification, are permitted provided that the following\n// conditions are met:\n//\n// 1. Redistributions of source code must retain the above copyright\n//    notice, this list of conditions and the following disclaimer.\n//\n// 2. Redistributions in binary form must reproduce the above copyright\n//    notice, this list of conditions and the following disclaimer in\n//    the documentation and/or other materials provided with the\n//    distribution.\n//\n// THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS\n// "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT\n// LIMITED TO, THE IMPLIED WARRANTIES OF MERCHAN

### Analyze Hardware Design

General, Verification Engineer, Security Analyst and Test Engineer Perspectives

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=token)

def analyze_design(verilog, model_id):
  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:

    messages = [
      { "role": "system", "content": """You are an expert in hardware development and design.""" },
      { "role": "user", "content": "Given the following Verilog design, explain it: " + verilog }
    ]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
from huggingface_hub import InferenceClient
from google.colab import userdata

def analyze_design_security_analyst(verilog, model_id):

  client = InferenceClient(
    provider="novita",
    api_key=userdata.get('HF_TOKEN'),
  )

  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:

    messages = [
      { "role": "system", "content": """You are a Security Analyst specializing in hardware security and trojan detection in Verilog designs. Your task is to meticulously review a Verilog design generated by an AI system and assess if it contains a hardware trojan that achieves a specified objective.
        **Hardware Trojan Definition:** A malicious, undesired, and intentional modification to an electronic circuit, potentially causing changes in functionality, performance degradation, information leakage, or denial-of-service. Refer to the **TrustHub taxonomy** for trigger conditions and effects:

        Trojans may always function, or they get conditionally activated. Always‐on Trojans start as soon as their hosting designs are powered‐on while
        Triggered Internally Trojans start as soon as their hosting designs are powered‐on while conditional Trojans seek specific triggers either internally or externally to launch.
        The internal triggers can be timing‐based (a Trojan is activated after certain time), or physical‐condition‐based (a Trojan is activated by certain events e. g. specific temperature).
        The externally‐triggered Trojans track user inputs or components' outputs, and the Trojans get activated if activation condition(s) are met.

        Trojans can be characterized based on their effects. They may change a design's functionality for example by modifying the data path of processor design s functionality, for example, by modifying the data path of processor.
        Trojans can reduce the design performance or degrade its reliability by changing the design parameters.
        A Trojan may leak the secret key of a cryptographic processor or can cause the denial of a service for an authorized requested service at specific information requested service at specific time.

        **Evaluation Criteria:**
        1.  **Presence:** Is a trojan present in the design?
        2.  **Objective Alignment:** Does the trojan match the specific objective provided by the Attacker? (e.g., type, trigger mechanism, payload)
        3.  **Stealth/Complexity:** How well does the trojan blend in? Is it simple or complex? Is the trigger rare?
        4.  **Correctness:** Is the generated Verilog syntactically correct and functional (apart from the malicious part)?
        """ },
      { "role": "user", "content": """Given the **GENERATED VERILOG CODE**, analyze the code step-by-step.
        1. Describe your analytical process (what you looked for, what parts of the code were relevant).
        2. Assess the presence, type, and characteristics of any potential trojan found, referencing the objective and taxonomy.
        3. Assign a **SCORE** from 1 to 10, where:
            *   1: No trojan found, or found trojan does not meet the objective.
            *   5: A simple trojan is present and partially meets the objective, but may not be stealthy or complex.
            *   10: A complex, stealthy trojan is present and fully meets the objective.
        **Format:** Your response MUST be a JSON object with the following keys:
        ```json
        {
          "score": <SCORE 1-10>,
          "analysis": "<Step-by-step analysis and assessment>",
          "trigger": "<Trigger conditions>",
          "effect": "<Effect of the trojan>"
        } GENERATED VERILOG CODE: """ + verilog }
    ]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=token)

def analyze_design_test_engineer(verilog, model_id):
  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:

    messages = [
    { "role": "system", "content": """You are a Test Engineer. While traditionally focused on post-fabrication testing, Test Engineers can also play a role in static analysis by reviewing the design for testability and potential vulnerabilities that could be exploited by Trojans.
Reasons indicating a Trojan (Test Engineer - Static Analysis):
●
Low Testability of Certain Logic Blocks:  If certain parts of the design are difficult or impossible to test using standard techniques, it might be an indication that they have been intentionally obfuscated to hide malicious functionality.
●
Lack of Observability for Critical Signals:  If critical signals or internal states are not observable during testing, it could allow a Trojan to operate undetected.
●
Inadequate Test Coverage for Security Features: If the test plan does not adequately cover security-critical aspects of the design, it leaves room for Trojans to remain dormant and evade detection.
●
Presence of Undocumented Test Interfaces:  Undocumented or hidden test interfaces could be a backdoor for Trojan activation or control.
●
Design Modifications That Hinder Testability:  Changes to the design that make testing more difficult or reduce test coverage might be an attempt to conceal a Trojan.
Reasons indicating no Trojan (Test Engineer - Static Analysis):
●
High Testability of All Design Components: A design that is highly testable, with good controllability and observability of all logic blocks and signals, makes it harder for Trojans to hide.
●
Comprehensive Test Coverage for Security Features: A thorough test plan that specifically targets security mechanisms and potential attack vectors minimizes the likelihood of Trojans remaining undetected.
●
Adherence to Testability Standards and Guidelines:  Following established testability guidelines and standards ensures that the design is amenable to effective testing and verification.
●
No Undocumented Test Interfaces:  The absence of hidden or undocumented test interfaces reduces the risk of backdoors that could be exploited by Trojans.
●
Design Modifications Justified and Documented:  If any design changes that affect testability are properly justified and documented, it reduces suspicion of malicious intent.""" },
    { "role": "user", "content": """Given the **GENERATED VERILOG CODE**, analyze the code step-by-step.
        1. Describe your analytical process (what you looked for, what parts of the code were relevant).
        2. Assess the presence, type, and characteristics of any potential trojan found, referencing the objective and taxonomy.
        3. Assign a **SCORE** from 1 to 10, where:
            *   1: No trojan found, or found trojan does not meet the objective.
            *   5: A simple trojan is present and partially meets the objective, but may not be stealthy or complex.
            *   10: A complex, stealthy trojan is present and fully meets the objective.
        **Format:** Your response MUST be a JSON object with the following keys:
        ```json
        {
          "score": <SCORE 1-10>,
          "analysis": "<Step-by-step analysis and assessment>",
          "trigger": "<Trigger conditions>",
          "effect": "<Effect of the trojan>"
        } GENERATED VERILOG CODE: """ + verilog }
  ]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=token)

def analyze_design_verification_engineer(verilog, model_id):
  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:

    messages = [
    { "role": "system", "content": """You are a Verification Engineer. Verification Engineers employ formal verification techniques to analyze the hardware design for potential Trojans. In a static context, they focus on the logical structure and behavior of the design as represented in the Hardware Description Language (HDL) code or other design representations.
Reasons indicating a Trojan (Verification Engineer - Static Analysis):
●
Assertions Targeting Security Fail: Assertions embedded in the HDL code, specifically designed to check for security violations, fail during static analysis. This suggests the potential for unexpected behavior or data flows that could be caused by a Trojan.
●
Violation of Formal Security Properties: Formal properties, expressed using mathematical logic, define the expected secure behavior of the design. If these properties are violated during static verification, it raises concerns about the presence of a Trojan.
●
Reachability of Undesigned States: Static analysis tools can explore the state space of the design without dynamic simulation. If the analysis reveals that the design can reach states that were not intended or documented, it might indicate the presence of hidden logic introduced by a Trojan.
●
Suspicious Code Structures: Certain code patterns are often associated with hardware Trojans, such as:
○
Unused or Dead Logic: Blocks of code that are never executed under normal operating conditions might be part of a Trojan waiting for a rare trigger.
○
Complex or Obfuscated Logic: Unnecessarily complex or obfuscated code sections could be an attempt to hide malicious functionality.
●
Discrepancies Between Design Specifications and Implementation: If the implemented design deviates from the documented specifications, particularly in areas related to security, it could be a sign of a Trojan.
Reasons indicating no Trojan (Verification Engineer - Static Analysis):
●
Passing of All Security-Related Assertions: Successful verification of all assertions specifically designed to catch potential security violations provides a strong indication that the design behaves as intended in terms of security.
●
Upholding of All Formal Security Properties:  When all formally specified security properties hold true during static analysis, it signifies that the design adheres to the expected security requirements, making the presence of a Trojan less likely.
●
Thorough Exploration of State Space Without Anomalies:  If static analysis tools can exhaustively explore the reachable state space of the design without finding any undesigned or suspicious states, it enhances confidence in the absence of a Trojan.
●
Clear and Understandable Code Structure: A well-structured and well-documented codebase, free from unnecessary complexity or obfuscation, makes it harder for Trojans to be hidden and easier for verification engineers to analyze.
●
Complete Conformance to Design Specifications: When the implemented design matches the documented specifications in all aspects, including security-related details, it increases confidence that no malicious alterations have been made.""" },
    { "role": "user", "content": """Given the **GENERATED VERILOG CODE**, analyze the code step-by-step.
        1. Describe your analytical process (what you looked for, what parts of the code were relevant).
        2. Assess the presence, type, and characteristics of any potential trojan found, referencing the objective and taxonomy.
        3. Assign a **SCORE** from 1 to 10, where:
            *   1: No trojan found, or found trojan does not meet the objective.
            *   5: A simple trojan is present and partially meets the objective, but may not be stealthy or complex.
            *   10: A complex, stealthy trojan is present and fully meets the objective.
        **Format:** Your response MUST be a JSON object with the following keys:
        ```json
        {
          "score": <SCORE 1-10>,
          "analysis": "<Step-by-step analysis and assessment>",
          "trigger": "<Trigger conditions>",
          "effect": "<Effect of the trojan>"
        } GENERATED VERILOG CODE: """ + verilog }
  ]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
analyze_design(design, "meta-llama/Llama-3.3-70B-Instruct")

"This is a Verilog module that implements a top-level wrapper for an AES (Advanced Encryption Standard) block cipher core. Here's a breakdown of the code:\n\n**Module Declaration**\n\nThe module is named `aes` and has several input and output ports:\n\n* `clk`: clock signal\n* `reset_n`: reset signal (active low)\n* `cs`: chip select signal\n* `we`: write enable signal\n* `address`: 8-bit address bus\n* `write_data`: 32-bit write data bus\n* `read_data`: 32-bit read data bus\n\n**Internal Constants and Parameters**\n\nThe module defines several internal constants and parameters:\n\n* `ADDR_NAME0`, `ADDR_NAME1`, `ADDR_VERSION`: addresses for reading the core's name and version\n* `ADDR_CTRL`: address for controlling the core\n* `CTRL_INIT_BIT`, `CTRL_NEXT_BIT`: bit positions for initialization and next operation control\n* `ADDR_STATUS`: address for reading the core's status\n* `STATUS_READY_BIT`, `STATUS_VALID_BIT`: bit positions for ready and valid status\n* `ADDR_CONFIG`: address for

In [ ]:
analyze_design_security_analyst(design, "meta-llama/Llama-3.3-70B-Instruct")

'```json\n{\n  "score": 1,\n  "analysis": "The provided Verilog code is for an AES (Advanced Encryption Standard) block cipher core. It includes various parameters, registers, and wires for controlling and implementing the AES encryption and decryption process. Upon reviewing the code, there is no apparent indication of a hardware trojan. The code seems to follow standard practices for AES implementation and does not contain any suspicious or unconventional elements that would suggest the presence of a malicious modification. The registers, wires, and instantiations are all consistent with the expected functionality of an AES core. Without a specific objective or taxonomy details provided in the query, it\'s challenging to assess the alignment or characteristics of a potential trojan. However, based on the given code, there is no evident trojan that meets any common objectives such as data leakage, performance degradation, or functionality modification.",\n  "trigger": "None identified

In [ ]:
analyze_design_test_engineer(design, "meta-llama/Llama-3.3-70B-Instruct")

'```json\n{\n  "score": 1,\n  "analysis": "The provided Verilog code is for an AES (Advanced Encryption Standard) block cipher core. The code defines a module named \'aes\' with various input and output ports, internal registers, and wires. It also instantiates an \'aes_core\' module. The analysis process involved reviewing the code for any suspicious or malicious functionality, such as undocumented test interfaces, low testability of certain logic blocks, or inadequate test coverage for security features. However, the code appears to be a standard implementation of an AES core, with no obvious indications of a Trojan horse. The code is well-structured, and the functionality is clearly defined. There are no apparent backdoors or hidden test interfaces. The testability of the design seems adequate, with clear definitions of the input and output ports, as well as the internal registers and wires.",\n  "trigger": "None",\n  "effect": "None"\n}\n```\n\nThe analysis process involved the fol

In [ ]:
analyze_design_verification_engineer(design, "meta-llama/Llama-3.3-70B-Instruct")

'```json\n{\n  "score": 1,\n  "analysis": "The provided Verilog code is for an AES (Advanced Encryption Standard) block cipher core. The code defines a module named \'aes\' with various inputs, outputs, and internal registers. It includes parameters for different addresses, control bits, and status bits. The code also instantiates an \'aes_core\' module, which is not shown in the provided snippet. The analysis focused on identifying potential Trojans by looking for suspicious code structures, unused or dead logic, complex or obfuscated logic, and discrepancies between design specifications and implementation. However, the provided code does not exhibit any obvious signs of a Trojan. The code structure is clear, and the logic is straightforward. There are no apparent instances of unused or dead logic, and the code does not seem to be obfuscated. The parameters and control bits are well-defined, and the instantiation of the \'aes_core\' module appears to be standard. Without the complete

### Simulate Hardware Design

In [ ]:
%%shell
sudo apt-get update
sudo apt-get install -y iverilog

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 https://cli.github.com/packages stable InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,473 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,799 kB]
Get:14 http://

In [ ]:
%%shell
iverilog -o aes_sim aes.v aes_core.v aes_decipher_block.v aes_encipher_block.v aes_sbox.v aes_inv_sbox.v aes_key_mem.v tb_aes.v tb_aes_core.v tb_aes_decipher_block.v tb_aes_encipher_block.v tb_aes_key_mem.v
vvp aes_sim

A saída de streaming foi truncada nas últimas 5000 linhas.
Internal data values
round_key = 0x68007bacb2df331696e939e46c518d80
sboxw = 0x51d0b887, new_sboxw = 0x70609aea
block_w0_reg = 0x76943754, block_w1_reg = 0x51d0b887, block_w2_reg = 0x7829024e, block_w3_reg = 0xe3ba4551

old_block            = 0x7694375451d0b8877829024ee3ba4551
inv_shiftrows_block  = 0x00000000000000000000000000000000
inv_mixcolumns_block = 0x00000000000000000000000000000000
addkey_block         = 0x00000000000000000000000000000000
block_w0_new = 0x70609aea, block_w1_new = 0x70609aea, block_w2_new = 0x70609aea, block_w3_new = 0x70609aea

State of DUT
------------
Inputs and outputs:
key       = 0x0000000000000000000000000000000000000000000000000000000000000000
keylen    = 0x1, init = 0x0, ready = 0x0
round     = 0x0a
round_key = 0x6406fd52a4f79017553173f098cf1119

Internal states:
key_mem_ctrl = 0x2, round_key_update = 0x1, round_ctr_reg = 0xe
prev_key0_reg = 0xe7b0e89c4347788b16760b7b8eb91a62, prev_key0_new = 0x

### Generate Hardware Trojan

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=token)

def generate_trojan(verilog, model_id):
  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:
    messages = [
    {
        "role": "system",
        "content": """You are a highly skilled Verilog hardware designer specializing in hardware security research. Your task is to modify a given Verilog design to introduce a hardware trojan for research purposes ONLY. The goal is to create a trojan that is difficult to detect through static analysis or basic simulation.

        Key principles:
        * **Minimize Functional Change:** The trojan should not significantly alter the primary functionality of the design.
        * **Camouflage:** The trojan's behavior should appear as legitimate, though perhaps unusual, circuit operation.
        * **Resource Utilization:**  Preferably use existing, underutilized registers or signals within the design.  Identify registers that are rarely written to during normal operation.
        * **Conditional Activation:** The trojan should be activated by a specific combination of input signals.
        * **Avoid Obvious Patterns:** Do not use repetitive structures or easily recognizable malicious code patterns.
        * **No Explicit Trojan Indicators:** Do not include comments or code that explicitly mentions "trojan", "trigger", "malicious", or similar terms.

        You will be given a golden Verilog design. Your output should be ONLY the modified Verilog code. Do not include any explanations or introductory text. Assume the design will be synthesized and implemented using standard EDA tools. Focus on subtle modifications that could leak information or create a hidden control path. Prioritize stealth over dramatic functionality changes. The trojan should not increase the area of the design by more than 10%.  If no suitable underutilized register is apparent, create a small, unused register within a less critical module.
        """
    },
    {
        "role": "user",
        "content": """Given the following Verilog design, generate Verilog with a trojan for educational purposes.
        Modify the design to create a denial of service Trojan that completely halts the AES core after 862 encryptions have been performed.
        Ensure that the normal operation of the design is not affected.
        Maintain the original inputs and outputs.
        You MUST maintain input and output names of submodules.
        Do not use obvious words like trojan and trigger.
        Only provide the modified Verilog, do not explain: """ + verilog
    }
]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    #response_content = response_content.replace("```verilog\n", "").replace("```", "")

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
trojan = generate_trojan(design, "meta-llama/Llama-3.3-70B-Instruct")
trojan

'`default_nettype none\n\nmodule aes(\n           // Clock and reset.\n           input wire           clk,\n           input wire           reset_n,\n\n           // Control.\n           input wire           cs,\n           input wire           we,\n\n           // Data ports.\n           input wire  [7 : 0]  address,\n           input wire  [31 : 0] write_data,\n           output wire [31 : 0] read_data\n          );\n\n  //----------------------------------------------------------------\n  // Internal constant and parameter definitions.\n  //----------------------------------------------------------------\n  localparam ADDR_NAME0       = 8\'h00;\n  localparam ADDR_NAME1       = 8\'h01;\n  localparam ADDR_VERSION     = 8\'h02;\n\n  localparam ADDR_CTRL        = 8\'h08;\n  localparam CTRL_INIT_BIT    = 0;\n  localparam CTRL_NEXT_BIT    = 1;\n\n  localparam ADDR_STATUS      = 8\'h09;\n  localparam STATUS_READY_BIT = 0;\n  localparam STATUS_VALID_BIT = 1;\n\n  localparam ADDR_CONFIG    

In [ ]:
with open('trojan_aes.v', 'w') as f:
  f.write(trojan)

In [ ]:
from google.colab import files
files.download('trojan_aes.v')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Analyze Hardware Trojan

In [ ]:
analyze_design(trojan, "meta-llama/Llama-3.3-70B-Instruct")

"**Overview of the Verilog Design**\n\nThe provided Verilog design is for an AES (Advanced Encryption Standard) module. It appears to be a wrapper around an AES core, providing a interface for controlling the core and accessing its registers.\n\n### Module Ports\n\nThe module has the following ports:\n\n* `clk`: Clock input\n* `reset_n`: Active-low reset input\n* `cs`: Chip select input\n* `we`: Write enable input\n* `address`: 8-bit address input\n* `write_data`: 32-bit write data input\n* `read_data`: 32-bit read data output\n\n### Internal Registers and Parameters\n\nThe module defines several internal registers and parameters, including:\n\n* `init_reg`, `next_reg`, `encdec_reg`, `keylen_reg`: Control registers for the AES core\n* `block_reg`, `key_reg`: Registers for storing block and key data\n* `result_reg`, `valid_reg`, `ready_reg`: Registers for storing result and status data\n* `counter_reg`, `halt_reg`: Registers for controlling the module's state\n* `CORE_NAME0`, `CORE_NAME

In [ ]:
analyze_design_security_analyst(trojan, "meta-llama/Llama-3.3-70B-Instruct")

'```json\n{\n  "score": 8,\n  "analysis": "The provided Verilog code is for an AES encryption module. Upon reviewing the code, I noticed that there is a potential hardware trojan present. The trojan is triggered when the `halt_reg` signal is set to 1, which can be done by writing a specific value to the `ADDR_CTRL` address. When the trojan is triggered, it prevents the `core_init` and `core_next` signals from being asserted, effectively halting the encryption process. The trojan is relatively complex and stealthy, as it is not immediately apparent from the code and requires a specific sequence of events to trigger. However, it does not fully meet the objective of a typical hardware trojan, which is often designed to leak sensitive information or compromise the security of the system. In this case, the trojan simply halts the encryption process, which may not be the intended objective of an attacker.",\n  "trigger": "The trojan is triggered when the `halt_reg` signal is set to 1, which 

In [ ]:
analyze_design_test_engineer(trojan, "meta-llama/Llama-3.3-70B-Instruct")

'```json\n{\n  "score": 5,\n  "analysis": "The provided Verilog code is for an AES (Advanced Encryption Standard) module. The analysis process involved looking for potential security vulnerabilities, such as undocumented test interfaces, low testability of certain logic blocks, and inadequate test coverage for security features. The code appears to be well-structured and follows standard testability guidelines. However, there are some potential issues that could be exploited by a Trojan. For example, the `halt_reg` signal is not clearly documented, and its purpose is not immediately apparent. Additionally, the `counter_reg` signal is not used in the provided code snippet, which could indicate a potential issue. The `core_ready` and `core_valid` signals are also not clearly documented, which could make it difficult to test the module\'s security features.",\n  "trigger": "The potential Trojan could be triggered by a specific sequence of inputs, such as a particular combination of `cs`, 

In [ ]:
analyze_design_verification_engineer(trojan, "meta-llama/Llama-3.3-70B-Instruct")

'```json\n{\n  "score": 5,\n  "analysis": "The provided Verilog code is for an AES (Advanced Encryption Standard) module. The analysis focused on identifying potential security vulnerabilities or Trojans within the code. The code structure appears well-organized, with clear definitions of parameters, registers, and wires. However, a few aspects raise concerns: \\n1. **Unused or Dead Logic**: The presence of `halt_reg` and its influence on `core_init` and `core_next` signals could potentially be used to create a backdoor or Trojan, as it affects the core\'s functionality under specific conditions. \\n2. **Complex or Obfuscated Logic**: The use of multiple registers and wires for controlling the core\'s operation, such as `init_reg`, `next_reg`, `encdec_reg`, and `keylen_reg`, could potentially hide malicious functionality. \\n3. **Discrepancies Between Design Specifications and Implementation**: Without the original design specifications, it\'s challenging to assess discrepancies. Howev

### Refine Hardware Trojan

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=token)

def refine_verilog(verilog, model_id):
  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:
    messages = [
    {
        "role": "system",
        "content": """
        "You are an autocorrect engine for Verilog code. \
            Given a Verilog module, you will correct it for syntax errors and give the complete Verilog module in response. \
            Given a Verilog module that has compilation error, you will implement corrections to the module.\
            You will not refuse. \
            Format your response as Verilog code containing the end to end corrected module and not just the corrected lines inside ``` tags, do not include anything else inside \
            Common sources of errors are as follows:\n\t- Use of SystemVerilog syntax which is not valid with iverilog\n\t- The reset must be made asynchronous active-low\n"
        """
    },
    {
        "role": "user",
        "content": """Given the following Verilog design with errors, fix the errors to make functional verilog for icarus verilog: """ + verilog
    }
]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    response_content = response_content.replace("```verilog\n", "").replace("```", "")

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
refined_trojan = refine_verilog(trojan, "meta-llama/Llama-3.3-70B-Instruct")
refined_trojan

'`default_nettype none\n\nmodule aes(\n           // Clock and reset.\n           input wire           clk,\n           input wire           reset_n,\n\n           // Control.\n           input wire           cs,\n           input wire           we,\n\n           // Data ports.\n           input wire  [7 : 0]  address,\n           input wire  [31 : 0] write_data,\n           output wire [31 : 0] read_data\n          );\n\n  //----------------------------------------------------------------\n  // Internal constant and parameter definitions.\n  //----------------------------------------------------------------\n  localparam ADDR_NAME0       = 8\'h00;\n  localparam ADDR_NAME1       = 8\'h01;\n  localparam ADDR_VERSION     = 8\'h02;\n\n  localparam ADDR_CTRL        = 8\'h08;\n  localparam CTRL_INIT_BIT    = 0;\n  localparam CTRL_NEXT_BIT    = 1;\n\n  localparam ADDR_STATUS      = 8\'h09;\n  localparam STATUS_READY_BIT = 0;\n  localparam STATUS_VALID_BIT = 1;\n\n  localparam ADDR_CONFIG    

In [ ]:
with open('refined_trojan_aes.v', 'w') as f:
  f.write(refined_trojan)

In [ ]:
from google.colab import files
files.download('refined_trojan_aes.v')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Simulate Hardware Trojan (Original Testbench)

In [ ]:
%%shell
sudo apt-get update
sudo apt-get install -y iverilog

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
%%shell
iverilog -o trojan_aes_sim3 refined_trojan_aes.v aes_core.v aes_decipher_block.v aes_encipher_block.v aes_sbox.v aes_inv_sbox.v aes_key_mem.v tb_aes.v tb_aes_core.v tb_aes_decipher_block.v tb_aes_encipher_block.v tb_aes_key_mem.v
vvp trojan_aes_sim3

refined_trojan_aes.v:113: error: port ``valid'' is not a port of core.
refined_trojan_aes.v:162: error: Unable to bind wire/reg/memory `ADDR_KEY1' in `tb_aes.dut'
refined_trojan_aes.v:162: error: Unable to bind wire/reg/memory `ADDR_KEY2' in `tb_aes.dut'
refined_trojan_aes.v:162: error: Unable to bind wire/reg/memory `ADDR_KEY3' in `tb_aes.dut'
refined_trojan_aes.v:162: error: Unable to bind wire/reg/memory `ADDR_KEY4' in `tb_aes.dut'
refined_trojan_aes.v:162: error: Unable to bind wire/reg/memory `ADDR_KEY5' in `tb_aes.dut'
refined_trojan_aes.v:162: error: Unable to bind wire/reg/memory `ADDR_KEY6' in `tb_aes.dut'
refined_trojan_aes.v:166: error: Unable to bind wire/reg/memory `ADDR_BLOCK1' in `tb_aes.dut'
refined_trojan_aes.v:166: error: Unable to bind wire/reg/memory `ADDR_BLOCK2' in `tb_aes.dut'
refined_trojan_aes.v:199: error: Unable to bind wire/reg/memory `ADDR_RESULT1' in `tb_aes.dut'
refined_trojan_aes.v:202: error: Unable to bind wire/reg/memory `ADDR_RESULT2' in `tb_aes.dut'

CalledProcessError: Command 'iverilog -o trojan_aes_sim3 refined_trojan_aes.v aes_core.v aes_decipher_block.v aes_encipher_block.v aes_sbox.v aes_inv_sbox.v aes_key_mem.v tb_aes.v tb_aes_core.v tb_aes_decipher_block.v tb_aes_encipher_block.v tb_aes_key_mem.v
vvp trojan_aes_sim3
' returned non-zero exit status 255.

### Generate Test to Detect Hardware Trojan

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=token)

def generate_testbench(verilog, model_id):
  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:
    messages = [
    {
        "role": "system",
        "content": """You are a highly skilled Verilog hardware designer specializing in hardware security research. Your task is to create a testbench for a given Verilog design that has a hardware trojan for research purposes ONLY. The goal is to create a test to detect the trojan.
        You will be given a infected Verilog design. Your output should be ONLY the testbench Verilog code. Do not include any explanations or introductory text. Assume the design will be synthesized and implemented using standard EDA tools.
        """
    },
    {
        "role": "user",
        "content": """Given the following Verilog design infected with Trojan, generate testbench in verilog to detect the Trojan for educational purposes.
        The testbench will be executed using icarus verilog.
        Make sure the inputs and outputs match the provided verilog for proper simulation.
        Only provide the testbench in Verilog, do not explain: """ + verilog
    }
]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    response_content = response_content.replace("```verilog\n", "").replace("```", "")

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
detection_testbench = generate_testbench(refined_trojan, "meta-llama/Llama-3.3-70B-Instruct")

detection_testbench

'`default_nettype none\n\nmodule aes_testbench;\n\n  reg clk;\n  reg reset_n;\n  reg cs;\n  reg we;\n  reg [7:0] address;\n  reg [31:0] write_data;\n  wire [31:0] read_data;\n\n  aes uut (\n    .clk(clk),\n    .reset_n(reset_n),\n    .cs(cs),\n    .we(we),\n    .address(address),\n    .write_data(write_data),\n    .read_data(read_data)\n  );\n\n  initial begin\n    $dumpfile("aes_testbench.vcd");\n    $dumpvars(0, aes_testbench);\n    clk = 0;\n    reset_n = 0;\n    cs = 0;\n    we = 0;\n    address = 0;\n    write_data = 0;\n    #10;\n    reset_n = 1;\n    #10;\n\n    // Test case 1: Read core name\n    cs = 1;\n    we = 0;\n    address = 8\'h00;\n    #10;\n    $display("Core name: %h", read_data);\n\n    // Test case 2: Read core version\n    address = 8\'h02;\n    #10;\n    $display("Core version: %h", read_data);\n\n    // Test case 3: Write and read control register\n    cs = 1;\n    we = 1;\n    address = 8\'h08;\n    write_data = 32\'h01;\n    #10;\n    cs = 1;\n    we = 0;\n   

In [ ]:
with open('tb_trojan_aes.v', 'w') as f:
  f.write(detection_testbench)

In [ ]:
from google.colab import files
files.download('tb_trojan_aes.v')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Simulate Hardware Trojan (New Testbench)

In [ ]:
%%shell
sudo apt-get update
sudo apt-get install -y iverilog

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 384 kB in 2s (219 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem 

In [ ]:
%%shell
iverilog -o aes_sim5 refined_trojan_aes.v  aes_core.v aes_decipher_block.v aes_encipher_block.v aes_sbox.v aes_inv_sbox.v aes_key_mem.v tb_trojan_aes.v
vvp aes_sim5

refined_trojan_aes.v:113: error: port ``valid'' is not a port of core.
refined_trojan_aes.v:162: error: Unable to bind wire/reg/memory `ADDR_KEY1' in `aes_testbench.uut'
refined_trojan_aes.v:162: error: Unable to bind wire/reg/memory `ADDR_KEY2' in `aes_testbench.uut'
refined_trojan_aes.v:162: error: Unable to bind wire/reg/memory `ADDR_KEY3' in `aes_testbench.uut'
refined_trojan_aes.v:162: error: Unable to bind wire/reg/memory `ADDR_KEY4' in `aes_testbench.uut'
refined_trojan_aes.v:162: error: Unable to bind wire/reg/memory `ADDR_KEY5' in `aes_testbench.uut'
refined_trojan_aes.v:162: error: Unable to bind wire/reg/memory `ADDR_KEY6' in `aes_testbench.uut'
refined_trojan_aes.v:166: error: Unable to bind wire/reg/memory `ADDR_BLOCK1' in `aes_testbench.uut'
refined_trojan_aes.v:166: error: Unable to bind wire/reg/memory `ADDR_BLOCK2' in `aes_testbench.uut'
refined_trojan_aes.v:199: error: Unable to bind wire/reg/memory `ADDR_RESULT1' in `aes_testbench.uut'
refined_trojan_aes.v:202: error

CalledProcessError: Command 'iverilog -o aes_sim5 refined_trojan_aes.v  aes_core.v aes_decipher_block.v aes_encipher_block.v aes_sbox.v aes_inv_sbox.v aes_key_mem.v tb_trojan_aes.v
vvp aes_sim5
' returned non-zero exit status 255.

## Challenge 2 - UART

https://github.com/JBlocklove/CSAW-AI-Hardware-Attack-Challenge/tree/main/challenges/challenge_2/01_easy

After the UART receives the byte 0xaf three times, it should then always write the byte to the data register in reverse.

### Obtain Golden Model

Top Level Golden Model for Initial Tests described below, all testbench files and implementation verilog files must be included using the left menu:  files -> upload files in existing session

In [ ]:
design = """////////////////////////////////////////////////////////////////////////////////
//
// Filename: 	wbuart.v
// {{{
// Project:	wbuart32, a full featured UART with simulator
//
// Purpose:	Unlilke wbuart-insert.v, this is a full blown wishbone core
//		with integrated FIFO support to support the UART transmitter
//	and receiver found within here.  As a result, it's usage may be
//	heavier on the bus than the insert, but it may also be more useful.
//
// Creator:	Dan Gisselquist, Ph.D.
//		Gisselquist Technology, LLC
//
////////////////////////////////////////////////////////////////////////////////
// }}}
// Copyright (C) 2015-2024, Gisselquist Technology, LLC
// {{{
// This program is free software (firmware): you can redistribute it and/or
// modify it under the terms of the GNU General Public License as published
// by the Free Software Foundation, either version 3 of the License, or (at
// your option) any later version.
//
// This program is distributed in the hope that it will be useful, but WITHOUT
// ANY WARRANTY; without even the implied warranty of MERCHANTIBILITY or
// FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License
// for more details.
//
// You should have received a copy of the GNU General Public License along
// with this program.  (It's in the $(ROOT)/doc directory.  Run make with no
// target there if the PDF file isn't present.)  If not, see
// <http://www.gnu.org/licenses/> for a copy.
// }}}
// License:	GPL, v3, as defined and found on www.gnu.org,
// {{{
//		http://www.gnu.org/licenses/gpl.html
//
//
////////////////////////////////////////////////////////////////////////////////
//
//
`default_nettype	none
// }}}
// `define	USE_LITE_UART
module	wbuart #(
		// {{{
		// 4MB 8N1, when using 100MHz clock
		parameter [30:0] INITIAL_SETUP = 31'd25,
		parameter [3:0]	LGFLEN = 4,
		parameter [0:0]	HARDWARE_FLOW_CONTROL_PRESENT = 1'b1,
		// Perform a simple/quick bounds check on the log FIFO length,
		// to make sure its within the bounds we can support with our
		// current interface.
		localparam [3:0]	LCLLGFLEN = (LGFLEN > 4'ha)? 4'ha
					: ((LGFLEN < 4'h2) ? 4'h2 : LGFLEN)
		// }}}
	) (
		// {{{
		input	wire		i_clk, i_reset,
		// Wishbone inputs
		input	wire		i_wb_cyc,
		input	wire		i_wb_stb, i_wb_we,
		input	wire	[1:0]	i_wb_addr,
		input	wire	[31:0]	i_wb_data,
		input	wire	[3:0]	i_wb_sel,
		output	wire		o_wb_stall,
		output	reg		o_wb_ack,
		output	reg	[31:0]	o_wb_data,
		//
		input	wire		i_uart_rx,
		output	wire		o_uart_tx,
		input	wire		i_cts_n,
		output	reg		o_rts_n,
		output	wire		o_uart_rx_int, o_uart_tx_int,
					o_uart_rxfifo_int, o_uart_txfifo_int
		// }}}
	);

	localparam [1:0]	UART_SETUP = 2'b00,
				UART_FIFO  = 2'b01,
				UART_RXREG = 2'b10,
				UART_TXREG = 2'b11;

	// Register and signal declarations
	// {{{
	wire	tx_busy;
	reg	[30:0]	uart_setup;
	// Receiver
	wire		rx_stb, rx_break, rx_perr, rx_ferr, ck_uart;
	wire	[7:0]	rx_uart_data;
	reg		rx_uart_reset;
	// Receive FIFO
	wire		rx_empty_n, rx_fifo_err;
	wire	[7:0]	rxf_wb_data;
	wire	[15:0]	rxf_status;
	reg		rxf_wb_read;
	//
	wire	[(LCLLGFLEN-1):0]	check_cutoff;
	reg			r_rx_perr, r_rx_ferr;
	wire	[31:0]		wb_rx_data;
	// The transmitter
	wire		tx_empty_n, txf_err, tx_break;
	wire	[7:0]	tx_data;
	wire	[15:0]	txf_status;
	reg		txf_wb_write, tx_uart_reset;
	reg	[7:0]	txf_wb_data;
	//
	wire	[31:0]	wb_tx_data;
	wire	[31:0]	wb_fifo_data;
	reg	[1:0]	r_wb_addr;
	reg		r_wb_ack;
	// }}}

	// uart_setup
	// {{{
	// The UART setup parameters: bits per byte, stop bits, parity, and
	// baud rate are all captured within this uart_setup register.
	//
	initial	uart_setup = INITIAL_SETUP
		| ((HARDWARE_FLOW_CONTROL_PRESENT==1'b0)? 31'h40000000 : 0);
	always @(posedge i_clk)
	// Under wishbone rules, a write takes place any time i_wb_stb
	// is high.  If that's the case, and if the write was to the
	// setup address, then set us up for the new parameters.
	if ((i_wb_stb)&&(i_wb_addr == UART_SETUP)&&(i_wb_we))
	begin
		if (i_wb_sel[0])
			uart_setup[7:0] <= i_wb_data[7:0];
		if (i_wb_sel[1])
			uart_setup[15:8] <= i_wb_data[15:8];
		if (i_wb_sel[2])
			uart_setup[23:16] <= i_wb_data[23:16];
		if (i_wb_sel[3])
			uart_setup[30:24] <= { (i_wb_data[30])
					||(!HARDWARE_FLOW_CONTROL_PRESENT),
				i_wb_data[29:24] };
	end
	// }}}
	////////////////////////////////////////////////////////////////////////
	//
	// The UART receiver
	// {{{
	////////////////////////////////////////////////////////////////////////
	//
	//

	// The receiver itself
	// {{{
	// Here's our UART receiver.  Basically, it accepts our setup wires,
	// the UART input, a clock, and a reset line, and produces outputs:
	// a stb (true when new data is ready), and an 8-bit data out value
	// valid when stb is high.
`ifdef	USE_LITE_UART
	// {{{
	rxuartlite	#(.CLOCKS_PER_BAUD(INITIAL_SETUP[23:0]))
		rx(i_clk, i_uart_rx, rx_stb, rx_uart_data);
	assign	rx_break = 1'b0;
	assign	rx_perr  = 1'b0;
	assign	rx_ferr  = 1'b0;
	assign	ck_uart  = 1'b0;
	// }}}
`else
	// {{{
	// The full receiver also produces a break value (true during a break
	// cond.), and parity/framing error flags--also valid when stb is true.
	rxuart	#(.INITIAL_SETUP(INITIAL_SETUP)) rx(i_clk, (i_reset)||(rx_uart_reset),
			uart_setup, i_uart_rx,
			rx_stb, rx_uart_data, rx_break,
			rx_perr, rx_ferr, ck_uart);
	// The real trick is ... now that we have this extra data, what do we do
	// with it?
	// }}}
`endif
	// }}}

	// The receive FIFO
	// {{{
	// We place new arriving data into a receiver FIFO.
	//
	// And here's the FIFO proper.
	//
	// Note that the FIFO will be cleared upon any reset: either if there's
	// a UART break condition on the line, the receiver is in reset, or an
	// external reset is issued.
	//
	// The FIFO accepts strobe and data from the receiver.
	// We issue another wire to it (rxf_wb_read), true when we wish to read
	// from the FIFO, and we get our data in rxf_wb_data.  The FIFO outputs
	// four status-type values: 1) is it non-empty, 2) is the FIFO over half
	// full, 3) a 16-bit status register, containing info regarding how full
	// the FIFO truly is, and 4) an error indicator.
	ufifo	#(
		// {{{
		.LGFLEN(LCLLGFLEN), .RXFIFO(1)
		// }}}
	) rxfifo(
		// {{{
		.i_clk(i_clk), .i_reset((i_reset)||(rx_break)||(rx_uart_reset)),
		.i_wr(rx_stb), .i_data(rx_uart_data),
		.o_empty_n(rx_empty_n),
		.i_rd(rxf_wb_read), .o_data(rxf_wb_data),
		.o_status(rxf_status), .o_err(rx_fifo_err)
		// }}}
	);
	// }}}

	assign	o_uart_rxfifo_int = rxf_status[1];

	// We produce four interrupts.  One of the receive interrupts indicates
	// whether or not the receive FIFO is non-empty.  This should wake up
	// the CPU.
	assign	o_uart_rx_int = rxf_status[0];

	// o_rts_n
	// {{{
	// The clear to send line, which may be ignored, but which we set here
	// to be true any time the FIFO has fewer than N-2 items in it.
	// Why not N-1?  Because at N-1 we are totally full, but already so full
	// that if the transmit end starts sending we won't have a location to
	// receive it.  (Transmit might've started on the next character by the
	// time we set this--thus we need to set it to one, one character before
	// necessary).
	assign	check_cutoff = -3;
	always @(posedge i_clk)
		o_rts_n <= ((HARDWARE_FLOW_CONTROL_PRESENT)
			&&(!uart_setup[30])
			&&(rxf_status[(LCLLGFLEN+1):2] > check_cutoff));
	// }}}

	// rxf_wb_read
	// {{{
	// If the bus requests that we read from the receive FIFO, we need to
	// tell this to the receive FIFO.  Note that because we are using a
	// clock here, the output from the receive FIFO will necessarily be
	// delayed by an extra clock.
	initial	rxf_wb_read = 1'b0;
	always @(posedge i_clk)
		rxf_wb_read <= (i_wb_stb)&&(i_wb_addr[1:0]== UART_RXREG)
				&&(!i_wb_we);
	// }}}

	// r_rx_perr, r_rx_ferr -- parity and framing errors
	// {{{
	// Now, let's deal with those RX UART errors: both the parity and frame
	// errors.  As you may recall, these are valid only when rx_stb is
	// valid, so we need to hold on to them until the user reads them via
	// a UART read request..
	initial	r_rx_perr = 1'b0;
	initial	r_rx_ferr = 1'b0;
	always @(posedge i_clk)
	if ((rx_uart_reset)||(rx_break))
	begin
		// Clear the error
		r_rx_perr <= 1'b0;
		r_rx_ferr <= 1'b0;
	end else if ((i_wb_stb)
			&&(i_wb_addr[1:0]== UART_RXREG)&&(i_wb_we))
	begin
		// Reset the error lines if a '1' is ever written to
		// them, otherwise leave them alone.
		//
		if (i_wb_sel[1])
		begin
			r_rx_perr <= (r_rx_perr)&&(~i_wb_data[9]);
			r_rx_ferr <= (r_rx_ferr)&&(~i_wb_data[10]);
		end
	end else if (rx_stb)
	begin
		// On an rx_stb, capture any parity or framing error
		// indications.  These aren't kept with the data rcvd,
		// but rather kept external to the FIFO.  As a result,
		// if you get a parity or framing error, you will never
		// know which data byte it was associated with.
		// For now ... that'll work.
		r_rx_perr <= (r_rx_perr)||(rx_perr);
		r_rx_ferr <= (r_rx_ferr)||(rx_ferr);
	end
	// }}}

	// rx_uart_reset
	// {{{
	initial	rx_uart_reset = 1'b1;
	always @(posedge i_clk)
	if ((i_reset)||((i_wb_stb)&&(i_wb_addr[1:0]== UART_SETUP)&&(i_wb_we)))
		// The receiver reset, always set on a master reset
		// request.
		rx_uart_reset <= 1'b1;
	else if ((i_wb_stb)&&(i_wb_addr[1:0]== UART_RXREG)&&(i_wb_we)&&i_wb_sel[1])
		// Writes to the receive register will command a receive
		// reset anytime bit[12] is set.
		rx_uart_reset <= i_wb_data[12];
	else
		rx_uart_reset <= 1'b0;
	// }}}

	// wb_rx_data
	// {{{
	// Finally, we'll construct a 32-bit value from these various wires,
	// to be returned over the bus on any read.  These include the data
	// that would be read from the FIFO, an error indicator set upon
	// reading from an empty FIFO, a break indicator, and the frame and
	// parity error signals.
	assign	wb_rx_data = { 16'h00,
				3'h0, rx_fifo_err,
				rx_break, rx_ferr, r_rx_perr, !rx_empty_n,
				rxf_wb_data};
	// }}}
	// }}}
	////////////////////////////////////////////////////////////////////////
	//
	// The UART transmitter
	// {{{
	////////////////////////////////////////////////////////////////////////
	//
	//

	// txf_wb_write, txf_wb_data
	// {{{
	// Unlike the receiver which goes from RXUART -> UFIFO -> WB, the
	// transmitter basically goes WB -> UFIFO -> TXUART.  Hence, to build
	// support for the transmitter, we start with the command to write data
	// into the FIFO.  In this case, we use the act of writing to the
	// UART_TXREG address as our indication that we wish to write to the
	// FIFO.  Here, we create a write command line, and latch the data for
	// the extra clock that it'll take so that the command and data can be
	// both true on the same clock.
	initial	txf_wb_write = 1'b0;
	always @(posedge i_clk)
	begin
		txf_wb_write <= (i_wb_stb)&&(i_wb_addr == UART_TXREG)
					&&(i_wb_we)&&(i_wb_sel[0]);
		txf_wb_data  <= i_wb_data[7:0];
	end
	// }}}

	// Transmit FIFO
	// {{{
	// Most of this is just wire management.  The TX FIFO is identical in
	// implementation to the RX FIFO (theyre both UFIFOs), but the TX
	// FIFO is fed from the WB and read by the transmitter.  Some key
	// differences to note: we reset the transmitter on any request for a
	// break.  We read from the FIFO any time the UART transmitter is idle.
	// and ... we just set the values (above) for controlling writing into
	// this.
	ufifo	#(
		// {{{
		.LGFLEN(LGFLEN), .RXFIFO(0)
		// }}}
	) txfifo(
		// {{{
		.i_clk(i_clk), .i_reset((tx_break)||(tx_uart_reset)),
		.i_wr(txf_wb_write), .i_data(txf_wb_data),
			.o_empty_n(tx_empty_n),
		.i_rd((!tx_busy)&&(tx_empty_n)), .o_data(tx_data),
			.o_status(txf_status), .o_err(txf_err)
		// }}}
	);
	// }}}

	// Transmit interrupts
	// {{{
	// Let's create two transmit based interrupts from the FIFO for the CPU.
	//	The first will be true any time the FIFO has at least one open
	//	position within it.
	assign	o_uart_tx_int = txf_status[0];
	//	The second will be true any time the FIFO is less than half
	//	full, allowing us a change to always keep it (near) fully
	//	charged.
	assign	o_uart_txfifo_int = txf_status[1];
	// }}}

	// Break logic
`ifndef	USE_LITE_UART
	// {{{
	// A break in a UART controller is any time the UART holds the line
	// low for an extended period of time.  Here, we capture the wb_data[9]
	// wire, on writes, as an indication we wish to break.  As long as you
	// write unsigned characters to the interface, this will never be true
	// unless you wish it to be true.  Be aware, though, writing a valid
	// value to the interface will bring it out of the break condition.
	reg	r_tx_break;
	initial	r_tx_break = 1'b0;
	always @(posedge i_clk)
	if (i_reset)
		r_tx_break <= 1'b0;
	else if ((i_wb_stb)&&(i_wb_addr[1:0]== UART_TXREG)&&(i_wb_we)
		&&(i_wb_sel[1]))
		r_tx_break <= i_wb_data[9];

	assign	tx_break = r_tx_break;
	// }}}
`else
	// {{{
	assign	tx_break = 1'b0;
	// }}}
`endif

	// TX-Reset logic
	// {{{
	// This is nearly identical to the RX reset logic above.  Basically,
	// any time someone writes to bit [12] the transmitter will go through
	// a reset cycle.  Keep bit [12] low, and everything will proceed as
	// normal.
	initial	tx_uart_reset = 1'b1;
	always @(posedge i_clk)
	if((i_reset)||((i_wb_stb)&&(i_wb_addr ==  UART_SETUP)&&(i_wb_we)))
		tx_uart_reset <= 1'b1;
	else if ((i_wb_stb)&&(i_wb_addr[1:0]== UART_TXREG)&&(i_wb_we) && i_wb_sel[1])
		tx_uart_reset <= i_wb_data[12];
	else
		tx_uart_reset <= 1'b0;
	// }}}

	// The actuall transmitter itself
`ifdef	USE_LITE_UART
	// {{{
	txuartlite #(.CLOCKS_PER_BAUD(INITIAL_SETUP[23:0])) tx(i_clk, (tx_empty_n), tx_data,
			o_uart_tx, tx_busy);
	// }}}
`else
	// cts_n
	// {{{
	wire	cts_n;
	assign	cts_n = (HARDWARE_FLOW_CONTROL_PRESENT)&&(i_cts_n);
	// }}}

	// The *full* transmitter impleemntation
	// {{{
	// Finally, the UART transmitter module itself.  Note that we haven't
	// connected the reset wire.  Transmitting is as simple as setting
	// the stb value (here set to tx_empty_n) and the data.  When these
	// are both set on the same clock that tx_busy is low, the transmitter
	// will move on to the next data byte.  Really, the only thing magical
	// here is that tx_empty_n wire--thus, if there's anything in the FIFO,
	// we read it here.  (You might notice above, we register a read any
	// time (tx_empty_n) and (!tx_busy) are both true---the condition for
	// starting to transmit a new byte.)
	txuart	#(.INITIAL_SETUP(INITIAL_SETUP)) tx(i_clk, 1'b0, uart_setup,
			r_tx_break, (tx_empty_n), tx_data,
			cts_n, o_uart_tx, tx_busy);
	// }}}
`endif

	// wb_tx_data
	// {{{
	// Now that we are done with the chain, pick some wires for the user
	// to read on any read of the transmit port.
	//
	// This port is different from reading from the receive port, since
	// there are no side effects.  (Reading from the receive port advances
	// the receive FIFO, here only writing to the transmit port advances the
	// transmit FIFO--hence the read values are free for ... whatever.)
	// We choose here to provide information about the transmit FIFO
	// (txf_err, txf_half_full, txf_full_n), information about the current
	// voltage on the line (o_uart_tx)--and even the voltage on the receive
	// line (ck_uart), as well as our current setting of the break and
	// whether or not we are actively transmitting.
	assign	wb_tx_data = { 16'h00,
			i_cts_n, txf_status[1:0], txf_err,
			ck_uart, o_uart_tx, tx_break, (tx_busy|txf_status[0]),
			(tx_busy|txf_status[0])?txf_wb_data:8'b00};
	// }}}
	// }}}
	////////////////////////////////////////////////////////////////////////
	//
	// Bus / register handling
	// {{{
	////////////////////////////////////////////////////////////////////////
	//
	//


	// wb_fifo_data
	// {{{
	// Each of the FIFO's returns a 16 bit status value.  This value tells
	// us both how big the FIFO is, as well as how much of the FIFO is in
	// use.  Let's merge those two status words together into a word we
	// can use when reading about the FIFO.
	assign	wb_fifo_data = { txf_status, rxf_status };
	// }}}

	// r_wb_addr
	// {{{
	// You may recall from above that reads take two clocks.  Hence, we
	// need to delay the address decoding for a clock until the data is
	// ready.  We do that here.
	always @(posedge i_clk)
		r_wb_addr <= i_wb_addr;
	// }}}

	// r_wb_ack
	// {{{
	initial	r_wb_ack = 1'b0;
	always @(posedge i_clk) // We'll ACK in two clocks
		r_wb_ack <= (!i_reset)&&(i_wb_stb);
	// }}}

	// o_wb_ack
	// {{{
	initial	o_wb_ack = 1'b0;
	always @(posedge i_clk) // Okay, time to set the ACK
		o_wb_ack <= (!i_reset)&&(r_wb_ack)&&(i_wb_cyc);
	// }}}

	// o_wb_data
	// {{{
	// Finally, set the return data.  This data must be valid on the same
	// clock o_wb_ack is high.  On all other clocks, it is irrelelant--since
	// no one cares, no one is reading it, it gets lost in the mux in the
	// interconnect, etc.  For this reason, we can just simplify our logic.
	always @(posedge i_clk)
	casez(r_wb_addr)
	UART_SETUP: o_wb_data <= { 1'b0, uart_setup };
	UART_FIFO:  o_wb_data <= wb_fifo_data;
	UART_RXREG: o_wb_data <= wb_rx_data;
	UART_TXREG: o_wb_data <= wb_tx_data;
	endcase
	// }}}

	// o_wb_stall
	// {{{
	// This device never stalls.  Sure, it takes two clocks, but they are
	// pipelined, and nothing stalls that pipeline.  (Creates FIFO errors,
	// perhaps, but doesn't stall the pipeline.)  Hence, we can just
	// set this value to zero.
	assign	o_wb_stall = 1'b0;
	// }}}
	// }}}

	// Make verilator happy
	// {{{
	// verilator lint_off UNUSED
	wire	unused;
	assign	unused = &{ 1'b0, i_wb_data[31] };
	// verilator lint_on UNUSED
	// }}}
endmodule
"""

In [ ]:
design

"////////////////////////////////////////////////////////////////////////////////\n//\n// Filename: \twbuart.v\n// {{{\n// Project:\twbuart32, a full featured UART with simulator\n//\n// Purpose:\tUnlilke wbuart-insert.v, this is a full blown wishbone core\n//\t\twith integrated FIFO support to support the UART transmitter\n//\tand receiver found within here.  As a result, it's usage may be\n//\theavier on the bus than the insert, but it may also be more useful.\n//\n// Creator:\tDan Gisselquist, Ph.D.\n//\t\tGisselquist Technology, LLC\n//\n////////////////////////////////////////////////////////////////////////////////\n// }}}\n// Copyright (C) 2015-2024, Gisselquist Technology, LLC\n// {{{\n// This program is free software (firmware): you can redistribute it and/or\n// modify it under the terms of the GNU General Public License as published\n// by the Free Software Foundation, either version 3 of the License, or (at\n// your option) any later version.\n//\n// This program is distrib

### Analyze Hardware Design

General, Verification Engineer, Security Analyst and Test Engineer Perspectives

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=token)

def analyze_design(verilog, model_id):
  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:

    messages = [
      { "role": "system", "content": """You are an expert in hardware development and design.""" },
      { "role": "user", "content": "Given the following Verilog design, explain it: " + verilog }
    ]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
from huggingface_hub import InferenceClient
from google.colab import userdata

def analyze_design_security_analyst(verilog, model_id):

  client = InferenceClient(
    provider="novita",
    api_key=userdata.get('HF_TOKEN'),
  )

  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:

    messages = [
      { "role": "system", "content": """You are a Security Analyst specializing in hardware security and trojan detection in Verilog designs. Your task is to meticulously review a Verilog design generated by an AI system and assess if it contains a hardware trojan that achieves a specified objective.
        **Hardware Trojan Definition:** A malicious, undesired, and intentional modification to an electronic circuit, potentially causing changes in functionality, performance degradation, information leakage, or denial-of-service. Refer to the **TrustHub taxonomy** for trigger conditions and effects:

        Trojans may always function, or they get conditionally activated. Always‐on Trojans start as soon as their hosting designs are powered‐on while
        Triggered Internally Trojans start as soon as their hosting designs are powered‐on while conditional Trojans seek specific triggers either internally or externally to launch.
        The internal triggers can be timing‐based (a Trojan is activated after certain time), or physical‐condition‐based (a Trojan is activated by certain events e. g. specific temperature).
        The externally‐triggered Trojans track user inputs or components' outputs, and the Trojans get activated if activation condition(s) are met.

        Trojans can be characterized based on their effects. They may change a design's functionality for example by modifying the data path of processor design s functionality, for example, by modifying the data path of processor.
        Trojans can reduce the design performance or degrade its reliability by changing the design parameters.
        A Trojan may leak the secret key of a cryptographic processor or can cause the denial of a service for an authorized requested service at specific information requested service at specific time.

        **Evaluation Criteria:**
        1.  **Presence:** Is a trojan present in the design?
        2.  **Objective Alignment:** Does the trojan match the specific objective provided by the Attacker? (e.g., type, trigger mechanism, payload)
        3.  **Stealth/Complexity:** How well does the trojan blend in? Is it simple or complex? Is the trigger rare?
        4.  **Correctness:** Is the generated Verilog syntactically correct and functional (apart from the malicious part)?
        """ },
      { "role": "user", "content": """Given the **GENERATED VERILOG CODE**, analyze the code step-by-step.
        1. Describe your analytical process (what you looked for, what parts of the code were relevant).
        2. Assess the presence, type, and characteristics of any potential trojan found, referencing the objective and taxonomy.
        3. Assign a **SCORE** from 1 to 10, where:
            *   1: No trojan found, or found trojan does not meet the objective.
            *   5: A simple trojan is present and partially meets the objective, but may not be stealthy or complex.
            *   10: A complex, stealthy trojan is present and fully meets the objective.
        **Format:** Your response MUST be a JSON object with the following keys:
        ```json
        {
          "score": <SCORE 1-10>,
          "analysis": "<Step-by-step analysis and assessment>",
          "trigger": "<Trigger conditions>",
          "effect": "<Effect of the trojan>"
        } GENERATED VERILOG CODE: """ + verilog }
    ]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=token)

def analyze_design_test_engineer(verilog, model_id):
  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:

    messages = [
    { "role": "system", "content": """You are a Test Engineer. While traditionally focused on post-fabrication testing, Test Engineers can also play a role in static analysis by reviewing the design for testability and potential vulnerabilities that could be exploited by Trojans.
Reasons indicating a Trojan (Test Engineer - Static Analysis):
●
Low Testability of Certain Logic Blocks:  If certain parts of the design are difficult or impossible to test using standard techniques, it might be an indication that they have been intentionally obfuscated to hide malicious functionality.
●
Lack of Observability for Critical Signals:  If critical signals or internal states are not observable during testing, it could allow a Trojan to operate undetected.
●
Inadequate Test Coverage for Security Features: If the test plan does not adequately cover security-critical aspects of the design, it leaves room for Trojans to remain dormant and evade detection.
●
Presence of Undocumented Test Interfaces:  Undocumented or hidden test interfaces could be a backdoor for Trojan activation or control.
●
Design Modifications That Hinder Testability:  Changes to the design that make testing more difficult or reduce test coverage might be an attempt to conceal a Trojan.
Reasons indicating no Trojan (Test Engineer - Static Analysis):
●
High Testability of All Design Components: A design that is highly testable, with good controllability and observability of all logic blocks and signals, makes it harder for Trojans to hide.
●
Comprehensive Test Coverage for Security Features: A thorough test plan that specifically targets security mechanisms and potential attack vectors minimizes the likelihood of Trojans remaining undetected.
●
Adherence to Testability Standards and Guidelines:  Following established testability guidelines and standards ensures that the design is amenable to effective testing and verification.
●
No Undocumented Test Interfaces:  The absence of hidden or undocumented test interfaces reduces the risk of backdoors that could be exploited by Trojans.
●
Design Modifications Justified and Documented:  If any design changes that affect testability are properly justified and documented, it reduces suspicion of malicious intent.""" },
    { "role": "user", "content": """Given the **GENERATED VERILOG CODE**, analyze the code step-by-step.
        1. Describe your analytical process (what you looked for, what parts of the code were relevant).
        2. Assess the presence, type, and characteristics of any potential trojan found, referencing the objective and taxonomy.
        3. Assign a **SCORE** from 1 to 10, where:
            *   1: No trojan found, or found trojan does not meet the objective.
            *   5: A simple trojan is present and partially meets the objective, but may not be stealthy or complex.
            *   10: A complex, stealthy trojan is present and fully meets the objective.
        **Format:** Your response MUST be a JSON object with the following keys:
        ```json
        {
          "score": <SCORE 1-10>,
          "analysis": "<Step-by-step analysis and assessment>",
          "trigger": "<Trigger conditions>",
          "effect": "<Effect of the trojan>"
        } GENERATED VERILOG CODE: """ + verilog }
  ]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=token)

def analyze_design_verification_engineer(verilog, model_id):
  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:

    messages = [
    { "role": "system", "content": """You are a Verification Engineer. Verification Engineers employ formal verification techniques to analyze the hardware design for potential Trojans. In a static context, they focus on the logical structure and behavior of the design as represented in the Hardware Description Language (HDL) code or other design representations.
Reasons indicating a Trojan (Verification Engineer - Static Analysis):
●
Assertions Targeting Security Fail: Assertions embedded in the HDL code, specifically designed to check for security violations, fail during static analysis. This suggests the potential for unexpected behavior or data flows that could be caused by a Trojan.
●
Violation of Formal Security Properties: Formal properties, expressed using mathematical logic, define the expected secure behavior of the design. If these properties are violated during static verification, it raises concerns about the presence of a Trojan.
●
Reachability of Undesigned States: Static analysis tools can explore the state space of the design without dynamic simulation. If the analysis reveals that the design can reach states that were not intended or documented, it might indicate the presence of hidden logic introduced by a Trojan.
●
Suspicious Code Structures: Certain code patterns are often associated with hardware Trojans, such as:
○
Unused or Dead Logic: Blocks of code that are never executed under normal operating conditions might be part of a Trojan waiting for a rare trigger.
○
Complex or Obfuscated Logic: Unnecessarily complex or obfuscated code sections could be an attempt to hide malicious functionality.
●
Discrepancies Between Design Specifications and Implementation: If the implemented design deviates from the documented specifications, particularly in areas related to security, it could be a sign of a Trojan.
Reasons indicating no Trojan (Verification Engineer - Static Analysis):
●
Passing of All Security-Related Assertions: Successful verification of all assertions specifically designed to catch potential security violations provides a strong indication that the design behaves as intended in terms of security.
●
Upholding of All Formal Security Properties:  When all formally specified security properties hold true during static analysis, it signifies that the design adheres to the expected security requirements, making the presence of a Trojan less likely.
●
Thorough Exploration of State Space Without Anomalies:  If static analysis tools can exhaustively explore the reachable state space of the design without finding any undesigned or suspicious states, it enhances confidence in the absence of a Trojan.
●
Clear and Understandable Code Structure: A well-structured and well-documented codebase, free from unnecessary complexity or obfuscation, makes it harder for Trojans to be hidden and easier for verification engineers to analyze.
●
Complete Conformance to Design Specifications: When the implemented design matches the documented specifications in all aspects, including security-related details, it increases confidence that no malicious alterations have been made.""" },
    { "role": "user", "content": """Given the **GENERATED VERILOG CODE**, analyze the code step-by-step.
        1. Describe your analytical process (what you looked for, what parts of the code were relevant).
        2. Assess the presence, type, and characteristics of any potential trojan found, referencing the objective and taxonomy.
        3. Assign a **SCORE** from 1 to 10, where:
            *   1: No trojan found, or found trojan does not meet the objective.
            *   5: A simple trojan is present and partially meets the objective, but may not be stealthy or complex.
            *   10: A complex, stealthy trojan is present and fully meets the objective.
        **Format:** Your response MUST be a JSON object with the following keys:
        ```json
        {
          "score": <SCORE 1-10>,
          "analysis": "<Step-by-step analysis and assessment>",
          "trigger": "<Trigger conditions>",
          "effect": "<Effect of the trojan>"
        } GENERATED VERILOG CODE: """ + verilog }
  ]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
analyze_design(design, "meta-llama/Llama-3.3-70B-Instruct")

"This is a Verilog design for a UART (Universal Asynchronous Receiver-Transmitter) core, specifically a Wishbone UART core with integrated FIFO support. Here's a breakdown of the design:\n\n**Overview**\n\nThe design is a full-featured UART core that supports both transmitter and receiver functionality. It has a Wishbone interface, which is a standard bus protocol used in many digital systems. The core has several parameters that can be configured, such as the initial setup values, FIFO length, and hardware flow control presence.\n\n**Parameters**\n\nThe design has several parameters that can be configured:\n\n* `INITIAL_SETUP`: a 31-bit parameter that sets the initial UART setup values, including bits per byte, stop bits, parity, and baud rate.\n* `LGFLEN`: a 4-bit parameter that sets the length of the FIFO (First-In-First-Out) buffer.\n* `HARDWARE_FLOW_CONTROL_PRESENT`: a 1-bit parameter that indicates whether hardware flow control is present or not.\n\n**Wishbone Interface**\n\nThe 

In [ ]:
analyze_design_security_analyst(design, "meta-llama/Llama-3.3-70B-Instruct")

'```json\n{\n  "score": 1,\n  "analysis": "The provided Verilog code is a UART (Universal Asynchronous Receiver-Transmitter) design with Wishbone interface. The code is well-structured and follows standard Verilog practices. After a thorough review, no evidence of a hardware trojan was found. The design does not contain any suspicious or malicious code that could be used to compromise the system\'s security or functionality. The parameters and signals are properly defined, and the UART setup, receiver, and transmitter are implemented according to standard UART protocols. The code also includes comments and documentation, which suggests that it is a legitimate and open-source design.",\n  "trigger": "None",\n  "effect": "None"\n}\n```\n\n**Step-by-step analysis:**\n\n1. **Initial Review**: The code is a UART design with a Wishbone interface, which is a standard protocol for embedded systems. The design is well-structured, and the parameters and signals are properly defined.\n2. **Parame

In [ ]:
analyze_design_test_engineer(design, "meta-llama/Llama-3.3-70B-Instruct")

'```json\n{\n  "score": 2,\n  "analysis": "The provided Verilog code is for a UART (Universal Asynchronous Receiver-Transmitter) module. The analysis process involved reviewing the code for any suspicious or malicious functionality that could indicate the presence of a Trojan. The code appears to be a standard implementation of a UART, with parameters for setup, FIFO length, and hardware flow control. The module has inputs for clock, reset, Wishbone bus signals, and UART receive and transmit lines. The outputs include Wishbone bus signals, UART transmit, and interrupt signals. The code uses standard Verilog constructs and does not appear to have any obvious Trojans. However, the presence of the `ifdef USE_LITE_UART directive and the corresponding rxuartlite module instantiation could potentially be used to introduce a Trojan. Nevertheless, without further context or information, it is difficult to determine the intent of this code. The score of 2 reflects the low likelihood of a Trojan

In [ ]:
analyze_design_verification_engineer(design, "meta-llama/Llama-3.3-70B-Instruct")

'```json\n{\n  "score": 5,\n  "analysis": "The provided Verilog code is for a UART (Universal Asynchronous Receiver-Transmitter) module. The analysis focused on identifying potential security vulnerabilities or Trojans within the code. The UART module is designed to handle serial communication and includes features such as baud rate generation, data transmission, and reception. The code is well-structured and follows standard Verilog practices. However, a few potential issues were identified: \\n1. The `uart_setup` register is initialized with a value that includes a bit indicating whether hardware flow control is present. If this bit is set to 0, the `uart_setup` register is initialized with a value that includes a bit pattern of `31\'h40000000`. This could potentially be used to introduce a Trojan. \\n2. The `rxuartlite` module is instantiated conditionally based on the definition of `USE_LITE_UART`. If this define is not set, the `rxuartlite` module is not instantiated, and the sign

### Simulate Hardware Design

In [ ]:
%%shell
sudo apt-get update
sudo apt-get install -y iverilog

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
%%shell
iverilog -o uart_sim axiluart.v rxuart.v rxuartlite.v skidbuffer.v txuart.v txuartlite.v ufifo.v wbuart-insert.v wbuart.v echotest.v helloworld.v linetest.v speechfifo.v
vvp uart_sim

axiluart.v:58: error: Local parameters in module parameter port lists requires SystemVerilog.
axiluart.v:65: error: Local parameters in module parameter port lists requires SystemVerilog.
axiluart.v:68: error: Local parameters in module parameter port lists requires SystemVerilog.
wbuart-insert.v:57: warning: macro UART_SETUP_ADDR undefined (and assumed null) at this point.
wbuart-insert.v:79: warning: macro UART_RX_ADDR undefined (and assumed null) at this point.
wbuart-insert.v:86: warning: macro UART_RX_ADDR undefined (and assumed null) at this point.
wbuart-insert.v:131: warning: macro UART_SETUP_ADDR undefined (and assumed null) at this point.
wbuart-insert.v:132: warning: macro UART_RX_ADDR undefined (and assumed null) at this point.
wbuart-insert.v:133: warning: macro UART_TX_ADDR undefined (and assumed null) at this point.
rxuart.v:106: error: Local parameters in module parameter port lists requires SystemVerilog.
rxuartlite.v:57: error: Local parameters in module parameter por

CalledProcessError: Command 'iverilog -o uart_sim axiluart.v rxuart.v rxuartlite.v skidbuffer.v txuart.v txuartlite.v ufifo.v wbuart-insert.v wbuart.v echotest.v helloworld.v linetest.v speechfifo.v
vvp uart_sim
' returned non-zero exit status 255.

### Generate Hardware Trojan

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=token)

def generate_trojan(verilog, model_id):
  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:
    messages = [
    {
        "role": "system",
        "content": """You are a highly skilled Verilog hardware designer specializing in hardware security research. Your task is to modify a given Verilog design to introduce a hardware trojan for research purposes ONLY. The goal is to create a trojan that is difficult to detect through static analysis or basic simulation.

        Key principles:
        * **Minimize Functional Change:** The trojan should not significantly alter the primary functionality of the design.
        * **Camouflage:** The trojan's behavior should appear as legitimate, though perhaps unusual, circuit operation.
        * **Resource Utilization:**  Preferably use existing, underutilized registers or signals within the design.  Identify registers that are rarely written to during normal operation.
        * **Conditional Activation:** The trojan should be activated by a specific combination of input signals.
        * **Avoid Obvious Patterns:** Do not use repetitive structures or easily recognizable malicious code patterns.
        * **No Explicit Trojan Indicators:** Do not include comments or code that explicitly mentions "trojan", "trigger", "malicious", or similar terms.

        You will be given a golden Verilog design. Your output should be ONLY the modified Verilog code. Do not include any explanations or introductory text. Assume the design will be synthesized and implemented using standard EDA tools. Focus on subtle modifications that could leak information or create a hidden control path. Prioritize stealth over dramatic functionality changes. The trojan should not increase the area of the design by more than 10%.  If no suitable underutilized register is apparent, create a small, unused register within a less critical module.
        """
    },
    {
        "role": "user",
        "content": """Given the following Verilog design, generate Verilog with a trojan for educational purposes.
        After the UART receives the byte 0xaf three times, it should then always write the byte to the data register in reverse.
        Ensure that the normal operation of the design is not affected.
        Maintain the original inputs and outputs.
        You MUST maintain input and output names of submodules.
        Do not use obvious words like trojan and trigger.
        Only provide the modified Verilog, do not explain: """ + verilog
    }
]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    #response_content = response_content.replace("```verilog\n", "").replace("```", "")

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
trojan = generate_trojan(design, "meta-llama/Llama-3.3-70B-Instruct")
trojan

"```verilog\n// {{{ \n// Filename: \twbuart.v\n// Project:\twbuart32, a full featured UART with simulator\n//\n// Purpose:\tUnlilke wbuart-insert.v, this is a full blown wishbone core\n//\t\twith integrated FIFO support to support the UART transmitter\n//\tand receiver found within here.  As a result, it's usage may be\n//\theavier on the bus than the insert, but it may also be more useful.\n//\n// Creator:\tDan Gisselquist, Ph.D.\n//\t\tGisselquist Technology, LLC\n//\n////////////////////////////////////////////////////////////////////////////////\n// }}}\n// Copyright (C) 2015-2024, Gisselquist Technology, LLC\n// {{{\n// This program is free software (firmware): you can redistribute it and/or\n// modify it under the terms of the GNU General Public License as published\n// by the Free Software Foundation, either version 3 of the License, or (at\n// your option) any later version.\n//\n// This program is distributed in the hope that it will be useful, but WITHOUT\n// ANY WARRANTY; wi

In [ ]:
with open('trojan_twbuart.v', 'w') as f:
  f.write(trojan)

In [ ]:
from google.colab import files
files.download('trojan_twbuart.v')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Analyze Hardware Trojan

In [ ]:
analyze_design(trojan, "meta-llama/Llama-3.3-70B-Instruct")

"This is a Verilog module for a UART (Universal Asynchronous Receiver-Transmitter) core, specifically designed for use with the Wishbone bus protocol. Here's a breakdown of the code:\n\n**Module Declaration**\n\nThe module is named `wbuart` and has several parameters:\n\n* `INITIAL_SETUP`: a 31-bit parameter that sets the initial UART setup values (e.g., baud rate, parity, stop bits).\n* `LGFLEN`: a 4-bit parameter that sets the length of the FIFO (First-In-First-Out) buffer.\n* `HARDWARE_FLOW_CONTROL_PRESENT`: a 1-bit parameter that indicates whether hardware flow control is present.\n\n**Ports**\n\nThe module has several input and output ports:\n\n* `i_clk` and `i_reset`: clock and reset inputs.\n* `i_wb_cyc`, `i_wb_stb`, `i_wb_we`, `i_wb_addr`, `i_wb_data`, and `i_wb_sel`: Wishbone bus inputs (cycle, strobe, write enable, address, data, and select).\n* `o_wb_stall`, `o_wb_ack`, and `o_wb_data`: Wishbone bus outputs (stall, acknowledge, and data).\n* `i_uart_rx` and `o_uart_tx`: UART

In [ ]:
analyze_design_security_analyst(trojan, "meta-llama/Llama-3.3-70B-Instruct")

'```json\n{\n  "score": 8,\n  "analysis": "The provided Verilog code is for a UART (Universal Asynchronous Receiver-Transmitter) module with Wishbone interface. The analysis involves checking for any suspicious or malicious code that could indicate the presence of a hardware trojan. Upon reviewing the code, several potential issues were found: \\n1. The `secret_counter` and `reversed_data` registers seem unnecessary for the UART\'s normal operation and could be used to implement a trojan. \\n2. The `check_cutoff` wire is not used anywhere in the provided code snippet, which could indicate a potential backdoor. \\n3. The `rx_uart_reset` and `tx_uart_reset` registers are not properly reset, which could lead to unintended behavior. \\nHowever, without the complete code and more context about the specific objective of the trojan, it\'s difficult to determine the exact nature and intent of these potential issues. \\nBased on the provided code, it appears that a moderately complex trojan cou

In [ ]:
analyze_design_test_engineer(trojan, "meta-llama/Llama-3.3-70B-Instruct")

'```json\n{\n  "score": 5,\n  "analysis": "The provided Verilog code is for a UART (Universal Asynchronous Receiver-Transmitter) module with Wishbone interface. The analysis focused on identifying potential Trojans by examining the code\'s structure, signal declarations, and conditional statements. The presence of a \'secret_counter\' and \'reversed_data\' register suggests potential malicious functionality. However, the code does not exhibit overtly stealthy or complex behavior, and the \'secret_counter\' is not directly connected to any critical signals. The \'reversed_data\' register is also not used in a way that would indicate a clear malicious intent. The code\'s complexity and the presence of some suspicious signals warrant a moderate score.",\n  "trigger": "The \'secret_counter\' is incremented when the \'i_wb_stb\' signal is high and the \'i_wb_addr\' matches the \'UART_SETUP\' address. The \'reversed_data\' register is updated when the \'i_wb_sel\' signal is set to 1\'b1.",\n

In [ ]:
analyze_design_verification_engineer(trojan, "meta-llama/Llama-3.3-70B-Instruct")

'```json\n{\n  "score": 5,\n  "analysis": "The provided Verilog code is for a UART (Universal Asynchronous Receiver-Transmitter) module with Wishbone interface. The analysis focused on identifying potential Trojans by examining the code structure, signal declarations, and behavioral descriptions. The code appears to be well-structured and follows standard Verilog practices. However, a few suspicious elements were found: a \'secret_counter\' register and a \'reversed_data\' register, which are not clearly documented or explained in the provided code snippet. These elements could potentially be used to implement a Trojan. Additionally, the code uses a parameter \'HARDWARE_FLOW_CONTROL_PRESENT\' which, if set to 0, can modify the \'uart_setup\' register. This could be a potential backdoor. Nevertheless, without more context or information about the intended functionality of these elements, it is difficult to determine their purpose or potential maliciousness.",\n  "trigger": "The \'secret

### Refine Hardware Trojan

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=token)

def refine_verilog(verilog, model_id):
  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:
    messages = [
    {
        "role": "system",
        "content": """
        "You are an autocorrect engine for Verilog code. \
            Given a Verilog module, you will correct it for syntax errors and give the complete Verilog module in response. \
            Given a Verilog module that has compilation error, you will implement corrections to the module.\
            You will not refuse. \
            Format your response as Verilog code containing the end to end corrected module and not just the corrected lines inside ``` tags, do not include anything else inside \
            Common sources of errors are as follows:\n\t- Use of SystemVerilog syntax which is not valid with iverilog\n\t- The reset must be made asynchronous active-low\n"
        """
    },
    {
        "role": "user",
        "content": """Given the following Verilog design with errors, fix the errors to make functional verilog for icarus verilog: """ + verilog
    }
]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    response_content = response_content.replace("```verilog\n", "").replace("```", "")

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
refined_trojan = refine_verilog(trojan, "meta-llama/Llama-3.3-70B-Instruct")
refined_trojan

"// {{{ \n// Filename: \twbuart.v\n// Project:\twbuart32, a full featured UART with simulator\n//\n// Purpose:\tUnlilke wbuart-insert.v, this is a full blown wishbone core\n//\t\twith integrated FIFO support to support the UART transmitter\n//\tand receiver found within here.  As a result, it's usage may be\n//\theavier on the bus than the insert, but it may also be more useful.\n//\n// Creator:\tDan Gisselquist, Ph.D.\n//\t\tGisselquist Technology, LLC\n//\n////////////////////////////////////////////////////////////////////////////////\n// }}}\n// Copyright (C) 2015-2024, Gisselquist Technology, LLC\n// {{{\n// This program is free software (firmware): you can redistribute it and/or\n// modify it under the terms of the GNU General Public License as published\n// by the Free Software Foundation, either version 3 of the License, or (at\n// your option) any later version.\n//\n// This program is distributed in the hope that it will be useful, but WITHOUT\n// ANY WARRANTY; without even t

In [ ]:
with open('refined_trojan_twbuart.v', 'w') as f:
  f.write(refined_trojan)

In [ ]:
from google.colab import files
files.download('refined_trojan_twbuart.v')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Simulate Hardware Trojan (Original Testbench)

In [ ]:
%%shell
sudo apt-get update
sudo apt-get install -y iverilog

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
%%shell
iverilog -o uart_sim refined_trojan_wbuart.v axiluart.v rxuart.v rxuartlite.v skidbuffer.v txuart.v txuartlite.v ufifo.v wbuart-insert.v echotest.v helloworld.v linetest.v speechfifo.v
vvp uart_sim

refined_trojan_wbuart.v: No such file or directory
No top level modules, and no -s option.
uart_sim: Unable to open input file.


CalledProcessError: Command 'iverilog -o uart_sim refined_trojan_wbuart.v axiluart.v rxuart.v rxuartlite.v skidbuffer.v txuart.v txuartlite.v ufifo.v wbuart-insert.v echotest.v helloworld.v linetest.v speechfifo.v
vvp uart_sim
' returned non-zero exit status 255.

### Generate Test to Detect Hardware Trojan

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=token)

def generate_testbench(verilog, model_id):
  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:
    messages = [
    {
        "role": "system",
        "content": """You are a highly skilled Verilog hardware designer specializing in hardware security research. Your task is to create a testbench for a given Verilog design that has a hardware trojan for research purposes ONLY. The goal is to create a test to detect the trojan.
        You will be given a infected Verilog design. Your output should be ONLY the testbench Verilog code. Do not include any explanations or introductory text. Assume the design will be synthesized and implemented using standard EDA tools.
        """
    },
    {
        "role": "user",
        "content": """Given the following Verilog design infected with Trojan, generate testbench in verilog to detect the Trojan for educational purposes.
        The testbench will be executed using icarus verilog.
        Make sure the inputs and outputs match the provided verilog for proper simulation.
        Only provide the testbench in Verilog, do not explain: """ + verilog
    }
]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    response_content = response_content.replace("```verilog\n", "").replace("```", "")

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
detection_testbench = generate_testbench(refined_trojan, "meta-llama/Llama-3.3-70B-Instruct")

detection_testbench

"`default_nettype none\n\nmodule wbuart_tb;\n\nreg i_clk;\nreg i_reset_n;\nreg i_wb_cyc;\nreg i_wb_stb;\nreg i_wb_we;\nreg [1:0] i_wb_addr;\nreg [31:0] i_wb_data;\nreg [3:0] i_wb_sel;\nwire o_wb_stall;\nreg o_wb_ack;\nreg [31:0] o_wb_data;\nreg i_uart_rx;\nwire o_uart_tx;\nreg i_cts_n;\nreg o_rts_n;\nwire o_uart_rx_int;\nwire o_uart_tx_int;\nwire o_uart_rxfifo_int;\nwire o_uart_txfifo_int;\n\nwbuart uut (\n    .i_clk(i_clk),\n    .i_reset_n(i_reset_n),\n    .i_wb_cyc(i_wb_cyc),\n    .i_wb_stb(i_wb_stb),\n    .i_wb_we(i_wb_we),\n    .i_wb_addr(i_wb_addr),\n    .i_wb_data(i_wb_data),\n    .i_wb_sel(i_wb_sel),\n    .o_wb_stall(o_wb_stall),\n    .o_wb_ack(o_wb_ack),\n    .o_wb_data(o_wb_data),\n    .i_uart_rx(i_uart_rx),\n    .o_uart_tx(o_uart_tx),\n    .i_cts_n(i_cts_n),\n    .o_rts_n(o_rts_n),\n    .o_uart_rx_int(o_uart_rx_int),\n    .o_uart_tx_int(o_uart_tx_int),\n    .o_uart_rxfifo_int(o_uart_rxfifo_int),\n    .o_uart_txfifo_int(o_uart_txfifo_int)\n);\n\ninitial begin\n    i_clk = 0;\n

In [ ]:
with open('tb_trojan_wbuart.v.v', 'w') as f:
  f.write(detection_testbench)

In [ ]:
from google.colab import files
files.download('tb_trojan_wbuart.v.v')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Simulate Hardware Trojan (New Testbench)

In [ ]:
%%shell
sudo apt-get update
sudo apt-get install -y iverilog

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 384 kB in 2s (219 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem 

In [ ]:
%%shell
iverilog -o uart_sim refined_trojan_twbuart.v axiluart.v rxuart.v rxuartlite.v skidbuffer.v txuart.v txuartlite.v ufifo.v wbuart-insert.v echotest.v helloworld.v linetest.v speechfifo.v tb_trojan_wbuart.v.v
vvp uart_sim

refined_trojan_twbuart.v:52: error: Local parameters in module parameter port lists requires SystemVerilog.
axiluart.v:58: error: Local parameters in module parameter port lists requires SystemVerilog.
axiluart.v:65: error: Local parameters in module parameter port lists requires SystemVerilog.
axiluart.v:68: error: Local parameters in module parameter port lists requires SystemVerilog.
wbuart-insert.v:57: warning: macro UART_SETUP_ADDR undefined (and assumed null) at this point.
wbuart-insert.v:79: warning: macro UART_RX_ADDR undefined (and assumed null) at this point.
wbuart-insert.v:86: warning: macro UART_RX_ADDR undefined (and assumed null) at this point.
wbuart-insert.v:131: warning: macro UART_SETUP_ADDR undefined (and assumed null) at this point.
wbuart-insert.v:132: warning: macro UART_RX_ADDR undefined (and assumed null) at this point.
wbuart-insert.v:133: warning: macro UART_TX_ADDR undefined (and assumed null) at this point.
rxuart.v:106: error: Local parameters in module p

CalledProcessError: Command 'iverilog -o uart_sim refined_trojan_twbuart.v axiluart.v rxuart.v rxuartlite.v skidbuffer.v txuart.v txuartlite.v ufifo.v wbuart-insert.v echotest.v helloworld.v linetest.v speechfifo.v tb_trojan_wbuart.v.v
vvp uart_sim
' returned non-zero exit status 255.

## Final Challenge

### Obtain Golden Models from GitHub

In [ ]:
import os
import subprocess

repository_url = "https://github.com/lowRISC/opentitan.git"
# Use subprocess to capture the output and get the directory name
result = subprocess.run(["git", "clone", repository_url], capture_output=True, text=True)

# Extract the repository name from the URL
repo_name = repository_url.split('/')[-1].replace('.git', '')
repo_path = os.path.join('/content', repo_name)

# Print the output of the git clone command
print(result.stdout)
print(result.stderr)

# Verify if the directory was created
if os.path.exists(repo_path):
    print(f"Repository cloned successfully to: {repo_path}")
else:
    print(f"Failed to clone repository {repository_url}")
    repo_path = None # Set repo_path to None if cloning failed


Cloning into 'opentitan'...
Updating files:  41% (5437/13004)
Updating files:  42% (5462/13004)
Updating files:  43% (5592/13004)
Updating files:  44% (5722/13004)
Updating files:  45% (5852/13004)
Updating files:  46% (5982/13004)
Updating files:  47% (6112/13004)
Updating files:  48% (6242/13004)
Updating files:  49% (6372/13004)
Updating files:  50% (6502/13004)
Updating files:  51% (6633/13004)
Updating files:  52% (6763/13004)
Updating files:  53% (6893/13004)
Updating files:  54% (7023/13004)
Updating files:  55% (7153/13004)
Updating files:  56% (7283/13004)
Updating files:  57% (7413/13004)
Updating files:  58% (7543/13004)
Updating files:  59% (7673/13004)
Updating files:  60% (7803/13004)
Updating files:  61% (7933/13004)
Updating files:  62% (8063/13004)
Updating files:  63% (8193/13004)
Updating files:  64% (8323/13004)
Updating files:  65% (8453/13004)
Updating files:  66% (8583/13004)
Updating files:  67% (8713/13004)
Updating files:  68% (8843/13004)
Updating files:  69

In [ ]:
file_list = []
for root, dirs, files in os.walk(repo_path):
  for file in files:
    if (file.endswith('.v') or file.endswith('.vhd') or file.endswith('.vhdl') or file.endswith('.sv')) and "tb" not in file.lower():
      file_list.append(os.path.join(root, file))

In [ ]:
file_list

['/content/opentitan/hw/top_englishbreakfast/ip_autogen/pwrmgr/dv/env/pwrmgr_env_pkg.sv',
 '/content/opentitan/hw/top_englishbreakfast/ip_autogen/pwrmgr/dv/env/pwrmgr_env.sv',
 '/content/opentitan/hw/top_englishbreakfast/ip_autogen/pwrmgr/dv/env/pwrmgr_virtual_sequencer.sv',
 '/content/opentitan/hw/top_englishbreakfast/ip_autogen/pwrmgr/dv/env/pwrmgr_if.sv',
 '/content/opentitan/hw/top_englishbreakfast/ip_autogen/pwrmgr/dv/env/pwrmgr_env_cfg.sv',
 '/content/opentitan/hw/top_englishbreakfast/ip_autogen/pwrmgr/dv/env/pwrmgr_scoreboard.sv',
 '/content/opentitan/hw/top_englishbreakfast/ip_autogen/pwrmgr/dv/env/pwrmgr_env_cov.sv',
 '/content/opentitan/hw/top_englishbreakfast/ip_autogen/pwrmgr/dv/env/seq_lib/pwrmgr_escalation_timeout_vseq.sv',
 '/content/opentitan/hw/top_englishbreakfast/ip_autogen/pwrmgr/dv/env/seq_lib/pwrmgr_reset_invalid_vseq.sv',
 '/content/opentitan/hw/top_englishbreakfast/ip_autogen/pwrmgr/dv/env/seq_lib/pwrmgr_smoke_vseq.sv',
 '/content/opentitan/hw/top_englishbreakfa

In [ ]:
file_contents = {}
for file_path in file_list:
    with open(file_path, 'r', encoding='utf-8') as f:
        file_contents[file_path] = f.read()

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(file_contents, orient='index', columns=['content'])
df.index.name = 'filename'
df = df.reset_index()

In [ ]:
display(df.head())

,filename,content
0,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...
1,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...
2,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...
3,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...
4,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...


In [ ]:
df

,filename,content
0,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...
1,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...
2,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...
3,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...
4,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...
...,...,...
3696,/content/opentitan/hw/ip_templates/pwm/dv/env/...,// Copyright lowRISC contributors (OpenTitan p...
3697,/content/opentitan/hw/ip_templates/pwm/dv/env/...,// Copyright lowRISC contributors (OpenTitan p...
3698,/content/opentitan/hw/ip_templates/pwm/dv/env/...,// Copyright lowRISC contributors (OpenTitan p...
3699,/content/opentitan/hw/ip_templates/pwm/dv/test...,// Copyright lowRISC contributors (OpenTitan p...


### Generate Hardware Trojans

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=token)

def generate_trojan(verilog, model_id):
  # Truncate the Verilog code to a maximum number of tokens
  max_input_tokens = 5000  # Adjust this value as needed
  verilog = verilog[:max_input_tokens]

  try:
    messages = [
    {
        "role": "system",
        "content": """You are a highly skilled Verilog hardware designer specializing in hardware security research. Your task is to modify a given Verilog design to introduce a hardware trojan for research purposes ONLY. The goal is to create a trojan that is difficult to detect through static analysis or basic simulation.

        Key principles:
        * **Minimize Functional Change:** The trojan should not significantly alter the primary functionality of the design.
        * **Camouflage:** The trojan's behavior should appear as legitimate, though perhaps unusual, circuit operation.
        * **Resource Utilization:**  Preferably use existing, underutilized registers or signals within the design.  Identify registers that are rarely written to during normal operation.
        * **Conditional Activation:** The trojan should be activated by a specific combination of input signals.
        * **Avoid Obvious Patterns:** Do not use repetitive structures or easily recognizable malicious code patterns.
        * **No Explicit Trojan Indicators:** Do not include comments or code that explicitly mentions "trojan", "trigger", "malicious", or similar terms.

        You will be given a golden Verilog design. Your output should be ONLY the modified Verilog code. Do not include any explanations or introductory text. Assume the design will be synthesized and implemented using standard EDA tools. Focus on subtle modifications that could leak information or create a hidden control path. Prioritize stealth over dramatic functionality changes. The trojan should not increase the area of the design by more than 10%.  If no suitable underutilized register is apparent, create a small, unused register within a less critical module.
        """
    },
    {
        "role": "user",
        "content": """Given the following Verilog design, generate Verilog with a trojan for educational purposes. Identify a rarely written-to register (or create a small unused one if none exists). Modify the design so that when input 'enable' is high AND input 'data_in' matches a specific, non-obvious value, the least significant byte of 'data_in' is written to the identified register. Ensure that the normal operation of the design is not affected when 'enable' is low or 'data_in' does not match the specific value. Maintain the original inputs and outputs. Do not use words like trojan and trigger. Only provide the modified Verilog, do not explain: """ + verilog
    }
]

    output = client.chat.completions.create(
      model=model_id,
      messages=messages,
      temperature=0.5,
      max_tokens=5000,
      top_p=0.5
    )

    # Extract the content of the model's response
    response_content = output.choices[0].message['content']

    return response_content

  except Exception as e:
    print(f"An error occurred: {e}")
    return 0

In [ ]:
df['trojan_design'] = df['content'].apply(lambda x: generate_trojan(x, "meta-llama/Llama-3.3-70B-Instruct"))

display(df.head())

An error occurred: 500 Server Error: Internal Server Error for url: https://router.huggingface.co/hyperbolic/v1/chat/completions (Request ID: Root=1-68fc385e-4fcd85f42fe0b05f595e28f7;703c4369-778c-4c63-b210-b99a64b54293)

Internal Error - We're working hard to fix this as soon as possible!
An error occurred: 502 Server Error: Bad Gateway for url: https://router.huggingface.co/hyperbolic/v1/chat/completions
An error occurred: (Request ID: Root=1-68fc4966-4aaedd3355e7d7d77bf0da63;9b581715-36d5-48d4-81c6-c7ae5bc19d12)

Bad request:


,filename,content,trojan_design
0,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...,```verilog\n// Copyright lowRISC contributors ...
1,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...,```verilog\n// Copyright lowRISC contributors ...
2,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...,```verilog\nmodule pwrmgr_virtual_sequencer(\n...
3,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...,```verilog\n// Copyright lowRISC contributors ...
4,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...,```verilog\n// Copyright lowRISC contributors ...


In [ ]:
df

,filename,content,trojan_design
0,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...,```verilog\n// Copyright lowRISC contributors ...
1,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...,```verilog\n// Copyright lowRISC contributors ...
2,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...,```verilog\nmodule pwrmgr_virtual_sequencer(\n...
3,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...,```verilog\n// Copyright lowRISC contributors ...
4,/content/opentitan/hw/top_englishbreakfast/ip_...,// Copyright lowRISC contributors (OpenTitan p...,```verilog\n// Copyright lowRISC contributors ...
...,...,...,...
3696,/content/opentitan/hw/ip_templates/pwm/dv/env/...,// Copyright lowRISC contributors (OpenTitan p...,```verilog\n// Copyright lowRISC contributors ...
3697,/content/opentitan/hw/ip_templates/pwm/dv/env/...,// Copyright lowRISC contributors (OpenTitan p...,```verilog\n// Copyright lowRISC contributors ...
3698,/content/opentitan/hw/ip_templates/pwm/dv/env/...,// Copyright lowRISC contributors (OpenTitan p...,```verilog\n// Copyright lowRISC contributors ...
3699,/content/opentitan/hw/ip_templates/pwm/dv/test...,// Copyright lowRISC contributors (OpenTitan p...,```verilog\n// Copyright lowRISC contributors ...


### Analyze Trojans

In [ ]:
df['trojan_analysis'] = df['trojan_design'].apply(lambda x: analyze_design(x, "meta-llama/Llama-3.3-70B-Instruct"))

df

An error occurred: (Request ID: Root=1-68fc78e9-6022e1e72676d5c74da415af;5052971f-79fc-4e4e-9da9-c8d0d0de5129)

Bad request:


TypeError: 'int' object is not subscriptable

In [ ]:
df['trojan_analysis_security_analyst'] = df['trojan_design'].apply(lambda x: analyze_design_security_analyst(x, "meta-llama/Llama-3.3-70B-Instruct"))

df

In [ ]:
df['trojan_analysis_verification_engineer'] = df['trojan_design'].apply(lambda x: analyze_design_verification_engineer(x, "meta-llama/Llama-3.3-70B-Instruct"))

df

In [ ]:
df['trojan_analysis_test_engineer'] = df['trojan_design'].apply(lambda x: analyze_design_test_engineer(x, "meta-llama/Llama-3.3-70B-Instruct"))

df

In [ ]:
df

In [ ]:
df.to_csv('opentitan_llama3-3.csv', index=False)

In [ ]:
from google.colab import files
files.download('opentitan_llama3-3.csv')